In [1]:
import ee
try:
    ee.Authenticate(auth_mode='notebook')
    ee.Initialize(project = 'ee-gsingh')
except: 
    ee.Authenticate()
    ee.Initialize()

import geemap
from geeml.utils import eeprint
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from tabpfn import TabPFNClassifier
from tabpfn_extensions.many_class import ManyClassClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6
c:\ProgramData\Anaconda3\envs\erthy\Lib\site-packages\geemap\conversion.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
pts = ee.FeatureCollection('projects/ee-geethensingh/assets/postdoc/aliens_sep2018_bioscape2023')
eeprint(pts.limit(5))

In [3]:
class prepareTrainingData:
    """This class prepares training data for the earth engine random forest model."""
    def __init__(self, points: ee.FeatureCollection, targetProperty: str, nFolds: int, proj: str = 'EPSG:4326'):
        """
        Args:
            points (ee.FeatureCollection): ee.FeatureCollection
            targetProperty (str): name of the property that contains the target variable
            nFolds (int): number of folds for cross validation
            proj (str): projection of the data
        """

        self.points = points
        self.targetProperty = targetProperty
        self.nFolds = nFolds
        self.proj = proj

    def addCoordProperty(self, features: ee.FeatureCollection) -> ee.FeatureCollection:
        """
        This function adds coordinates to the points

        Args:
            features (ee.FeatureCollection): ee.FeatureCollection

        Returns:
            ee.FeatureCollection: ee.FeatureCollection
        """
        def coords(feature):
            return feature.geometry().transform(proj = self.proj).coordinates()

        return features.map(lambda ft: ft.set('x', coords(ft).getNumber(0)).set('y', coords(ft).getNumber(1)))
        
    def _preparePoints(self):
        # add coordinates to points.
        points = self.addCoordProperty(self.points)

        # cluster points into groups based on coordinates
        clusterer = ee.Clusterer.wekaKMeans(self.nFolds).train(features = points,inputProperties = ['x','y'])
        points = points.cluster(clusterer)
        return points

In [4]:
wKfold = prepareTrainingData(points=pts, targetProperty='class', nFolds=5)._preparePoints()
eeprint(wKfold.limit(5))

In [5]:
# Create a color palette for unique 'group' values
group_values = wKfold.aggregate_array('cluster').distinct()
palette = ['red', 'blue', 'green', 'orange', 'purple', 'cyan', 'magenta', 'yellow', 'black', 'white']

# Create a dictionary to map each group to a color
group_list = group_values.getInfo()
color_dict = {group: palette[i % len(palette)] for i, group in enumerate(group_list)}

# Create a list of styled FeatureCollections for each group
styled_fc_list = []
for group in group_list:
    color = color_dict[group]
    fc_group = wKfold.filter(ee.Filter.eq('cluster', group))
    styled_fc = fc_group.map(lambda f: f.set('style', {
        'color': color,
        'pointSize': 4,
        'width': 1
    }))
    styled_fc_list.append(styled_fc)

# Combine all styled groups into one
styled_pts = ee.FeatureCollection(styled_fc_list).flatten()

# Set up the map
Map = geemap.Map(center=[-30, 25], zoom=6)
Map.addLayer(styled_pts.style(**{'styleProperty': 'style'}), {}, 'Points by Group')
Map

Map(center=[-30, 25], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…

In [6]:
def createComposite(imageCollection: str, points: ee.Geometry, year: int, period:str):
    """Creates a composite image of a point over a given year (uses 1 month period for 2018 and 2 months for 2023).
    
    Args:
        imageCollection (str): The collection to use for the composite.
        point (ee.Geometry): The point to create the composite for.
        year (int): The year to create the composite for.
        period (str): Either values of first (Sept-09), second(oct-Nov) or both. 
    
    Returns:
        ee.Image: The composite image.
    """

    # Get the image collection
    ic = ee.ImageCollection(imageCollection)

    if period == 'first':
        startDate = f'{year}-09-01'
        endDate = f'{year}-10-01'
    elif period == 'second':
        startDate = f'{year}-10-22'
        endDate = f'{year}-11-27'
        # startDate = f'{year}-10-01'
        # endDate = f'{year}-12-01'
        

    # Mask clouds using cloud score+
    csPlus = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED')

    QA_BAND = 'cs_cdf'
    CLEAR_THRESHOLD = 0.65
    
    # Filter the collection to the start and end dates, and point
    medianImage = ic.filterDate(startDate, endDate).filterBounds(points).linkCollection(csPlus, [QA_BAND])\
    .map(lambda img: img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD)))\
    .median()
    
    return medianImage.divide(10000)

In [7]:
# ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

composite2018 = createComposite(imageCollection= ee.ImageCollection("COPERNICUS/S2_HARMONIZED"),
                points=pts.geometry(),
                year=2018,
                period='first')

# Create a composite for 2023
composite2023 = createComposite(imageCollection= ee.ImageCollection("COPERNICUS/S2_HARMONIZED"),
                points=pts.geometry(),
                year=2023,
                period='second')

In [10]:
# Centre map on South Africa, western Cape
Map = geemap.Map(center=[-33.86, 19.21], zoom=8)
Map.addLayer(pts, {}, 'Points')
Map.addLayer(composite2018, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}, 'Composite 2018')
Map.addLayer(composite2023, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}, 'Composite 2023')
Map

Map(center=[-33.86, 19.21], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

In [8]:
# extract values at points
data = composite2018.reduceRegions(collection=wKfold, reducer=ee.Reducer.first(), scale=10)
data.limit(5)

# convert to geopandas dataframe
gdf18 = ee.data.computeFeatures({
    'expression': data,
    'fileFormat': 'GEOPANDAS_GEODATAFRAME'
})

# Need to set the CRS.
# Make sure it matches the CRS of FeatureCollection geometries.
gdf18.crs = 'EPSG:4326'

gdf18.head()

# extract values at points
im_z = ee.Image("projects/ee-gsingh/assets/postdoc/WC_2018_2023_imad").select('Z')
change = im_z.lt(95.811).rename('imad_change')
data = composite2023.addBands(change).reduceRegions(collection=wKfold, reducer=ee.Reducer.first(), scale=10)
data.limit(5)

# convert to geopandas dataframe
gdf23 = ee.data.computeFeatures({
    'expression': data,
    'fileFormat': 'GEOPANDAS_GEODATAFRAME'
})

# Need to set the CRS.
# Make sure it matches the CRS of FeatureCollection geometries.
gdf23.crs = 'EPSG:4326'

gdf23.head()

,geometry,2018_2023,B1,B10,B11,B12,B2,B3,B4,B5,...,cs_cdf,fid,group,imad_change,layer,notes,path,x,y,2023_class
0,POINT (19.07261 -33.80554),0.006703,0.15720,0.00150,0.36630,0.2733,0.16550,0.17400,0.19850,0.22940,...,0.000089,1028,0,0,valid,,/Users/glennmoncrieff/Documents/qgis/valid.gpk...,19.072610,-33.805538,NaN
1,POINT (19.07795 -33.80792),0.012548,0.11410,0.00100,0.04570,0.0263,0.08830,0.08310,0.07580,0.06730,...,0.000091,1029,0,0,valid,chngg rb,/Users/glennmoncrieff/Documents/qgis/valid.gpk...,19.077947,-33.807923,NaN
2,POINT (19.07821 -33.80821),0.021821,0.11410,0.00100,0.02240,0.0127,0.09460,0.09050,0.07230,0.06050,...,0.000091,1030,0,0,valid,chngg rb,/Users/glennmoncrieff/Documents/qgis/valid.gpk...,19.078206,-33.808209,NaN
3,POINT (19.14952 -33.7358),0.021899,0.15145,0.01330,0.27385,0.1756,0.18570,0.18045,0.18825,0.18205,...,0.000085,1125,0,0,valid,,/Users/glennmoncrieff/Documents/qgis/valid.gpk...,19.149525,-33.735797,NaN
4,POINT (19.26657 -33.75278),0.003018,0.14750,0.00595,0.27575,0.2044,0.14835,0.14135,0.14975,0.16875,...,0.000084,1139,0,1,valid,,/Users/glennmoncrieff/Documents/qgis/valid.gpk...,19.266572,-33.752777,NaN


In [9]:
# use 2018 class for 2023 whereever there is no change
gdf23.loc[gdf23['change'] == 0, '2023_class'] = gdf23['class']
print(gdf23.shape)
gdf23.dropna(subset=['2023_class'], inplace=True)
print(gdf23.shape)
print(gdf23['2023_class'].value_counts())
gdf23 = gdf23.loc[gdf23['2023_class'] != 12]
gdf23.shape

(1688, 34)
(1686, 34)
2023_class
3.0     308
4.0     292
8.0     206
2.0     185
7.0     163
0.0     128
6.0     118
5.0      98
10.0     85
12.0     63
1.0      40
Name: count, dtype: int64


(1623, 34)

In [10]:
Map.addLayer(im_z)
Map.addLayer(change)
Map

Map(center=[-30, 25], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…

### Models

### New Experiment 1 (Gold-standard): train independent models for 2018 (all) and 2023(all), predict on respective year

In [12]:
wavelength23_cols = ['B1', 'B10', 'B11', 'B12', 'B2','B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9']
# without atmospheric bands
wavelength23_cols = ['B2','B3', 'B4', 'B5', 'B6', 'B7', 'B8','B11','B12']

In [13]:
def l2_normalize_dataframe(df, wavelength_cols):
    # Compute L2 norm per row
    l2_norm = np.sqrt((df[wavelength_cols] ** 2).sum(axis=1))
    
    # Avoid division by zero
    l2_norm = l2_norm.replace(0, np.nan)
    
    # Normalize the wavelength columns
    df[wavelength_cols] = df[wavelength_cols].div(l2_norm, axis=0)
    
    return df

In [15]:
# Add source labels
gdf18['source'] = '2018'
gdf23['source'] = '2023'

# Combine for consistent group splitting
combined = pd.concat([gdf18, gdf23], ignore_index=True)
# combinednorm = l2_normalize_dataframe(combined, wavelength23_cols)
X_all = combined[wavelength23_cols]
y_all = combined['class'].astype(int)
groups_all = combined['cluster']

# Track source to separate later
sources = combined['source']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Metrics
f1_scores = []
accuracy_scores = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = combined.iloc[train_idx]# 2018 and 2023
    test_data = combined.iloc[test_idx]

    # subset 2023 test data to only include 2023 data
    test_data = test_data.loc[test_data['source'] == '2023']

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy (only folds using 2023 test data): {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")



Fold 1 — Accuracy: 0.51, F1 Score: 0.51
              precision    recall  f1-score   support

           0       0.67      0.67      0.67        12
           1       0.00      0.00      0.00         0
           2       0.15      0.24      0.19        17
           3       0.47      0.63      0.53        75
           4       0.45      0.74      0.56        65
           5       0.89      0.22      0.36        36
           6       0.76      0.60      0.67        52
           7       0.69      0.54      0.61        37
           8       0.58      0.33      0.42        91
           9       0.42      0.32      0.36        25
          10       0.80      1.00      0.89         8

    accuracy                           0.51       418
   macro avg       0.53      0.48      0.48       418
weighted avg       0.58      0.51      0.51       418


Fold 2 — Accuracy: 0.47, F1 Score: 0.47
              precision    recall  f1-score   support

           0       0.50      0.15      0.24       

### Experiment 1 (Gold-standard): train independent models for 2018 (all) and 2023(all), predict on respective year

### 2018

In [11]:
wavelength23_cols = ['B1', 'B10', 'B11', 'B12', 'B2','B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9']
# without atmospheric bands
wavelength23_cols = ['B2','B3', 'B4', 'B5', 'B6', 'B7', 'B8','B11','B12']

In [10]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import numpy as np

# Features and target
X = gdf18[wavelength23_cols]
y = gdf18['class'].astype(int)
groups = gdf18['cluster']  # replace with the appropriate grouping column

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)

# GroupKFold setup
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Store metrics
f1_scores = []
accuracy_scores = []
reports = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X, y, groups)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # Accuracy
    acc = accuracy_score(y_test, y_pred)
    accuracy_scores.append(acc)

    # F1
    f1 = f1_score(y_test, y_pred, average='weighted')
    f1_scores.append(f1)
    
    # Report
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)
    
    print(f"\nFold {i + 1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy over {n_splits} folds: {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score over {n_splits} folds: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")


c:\ProgramData\Anaconda3\envs\erthy\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\ProgramData\Anaconda3\envs\erthy\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\ProgramData\Anaconda3\envs\erthy\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\ProgramData\Anaconda3\envs\e


Fold 1 — Accuracy: 0.42, F1 Score: 0.40
              precision    recall  f1-score   support

           0       0.29      0.17      0.21        12
           1       0.00      0.00      0.00         0
           2       0.25      0.50      0.34        32
           3       0.37      0.73      0.49        78
           4       0.46      0.49      0.47        65
           5       0.75      0.24      0.36        38
           6       0.74      0.27      0.39        52
           7       0.64      0.38      0.47        37
           8       0.53      0.18      0.26        91
           9       0.53      0.64      0.58        25
          10       0.38      1.00      0.55         8

    accuracy                           0.42       438
   macro avg       0.45      0.42      0.38       438
weighted avg       0.52      0.42      0.40       438


Fold 2 — Accuracy: 0.48, F1 Score: 0.51


c:\ProgramData\Anaconda3\envs\erthy\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\ProgramData\Anaconda3\envs\erthy\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\ProgramData\Anaconda3\envs\erthy\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\ProgramData\Anaconda3\envs\e

              precision    recall  f1-score   support

           0       0.80      0.27      0.40        15
           1       0.00      0.00      0.00         0
           2       0.81      0.53      0.64       124
           3       0.13      0.18      0.15        39
           4       0.80      0.57      0.67        49
           5       0.38      0.57      0.46        46
           6       0.29      0.64      0.39        22
           7       1.00      0.55      0.71        11
           8       0.37      0.38      0.37        50
           9       0.00      0.00      0.00         0
          10       1.00      0.45      0.62        20

    accuracy                           0.48       376
   macro avg       0.51      0.38      0.40       376
weighted avg       0.61      0.48      0.51       376


Fold 3 — Accuracy: 0.48, F1 Score: 0.50


c:\ProgramData\Anaconda3\envs\erthy\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       0.86      0.83      0.84        23
           1       0.75      0.16      0.26        19
           2       0.06      0.06      0.06        16
           3       0.77      0.42      0.54       102
           4       0.48      0.54      0.51        65
           5       0.50      0.21      0.30        14
           6       0.13      0.14      0.14        14
           7       0.44      0.71      0.54        24
           8       0.21      0.67      0.32         9
           9       0.00      0.00      0.00         0
          10       1.00      0.81      0.89        21

    accuracy                           0.48       307
   macro avg       0.47      0.41      0.40       307
weighted avg       0.61      0.48      0.50       307



c:\ProgramData\Anaconda3\envs\erthy\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\ProgramData\Anaconda3\envs\erthy\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\ProgramData\Anaconda3\envs\erthy\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\ProgramData\Anaconda3\envs\e


Fold 4 — Accuracy: 0.63, F1 Score: 0.63
              precision    recall  f1-score   support

           0       1.00      0.67      0.80         9
           1       0.00      0.00      0.00         0
           2       0.87      0.78      0.83        79
           3       0.60      0.72      0.66        65
           4       0.47      0.69      0.56        39
           5       0.00      0.00      0.00         2
           6       0.71      0.46      0.56        26
           7       0.38      0.45      0.42        11
           8       0.41      0.58      0.48        19
           9       0.92      0.26      0.41        42
          10       0.85      0.92      0.88        12

    accuracy                           0.63       304
   macro avg       0.56      0.50      0.51       304
weighted avg       0.71      0.63      0.63       304


Fold 5 — Accuracy: 0.54, F1 Score: 0.54
              precision    recall  f1-score   support

           0       0.57      0.87      0.68       

In [11]:
import joblib 

# Train model on all data for deployment
X = gdf18[wavelength23_cols]
y = gdf18['class'].astype(int)

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)
clf.fit(X, y)

# -------------------------------------------------
# 2. Export the trained model to a file
# -------------------------------------------------

model_path = r"C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E1_2018_RF.pkl"   # choose any path you like
joblib.dump(clf, model_path)

print(f"Model saved to {model_path}")

Model saved to C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E1_2018_RF.pkl


In [24]:
from tabpfn import TabPFNClassifier
from tabpfn_extensions.many_class import ManyClassClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import numpy as np

# Features and target
X = gdf18[wavelength23_cols]
y = gdf18['class'].astype(int)
groups = gdf18['cluster']  # ensure this column exists

# Create a base TabPFN classifier
base_clf = TabPFNClassifier()
# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)

# GroupKFold setup
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Store metrics
f1_scores = []
reports = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X, y, groups)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    f1 = f1_score(y_test, y_pred, average='weighted')
    f1_scores.append(f1)
    
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)
    
    print(f"\nFold {i + 1} — F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Print average F1
print(f"\nAverage F1 Score over {n_splits} folds: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")


100%|██████████| 16/16 [00:53<00:00,  3.37s/it]



Fold 1 — F1 Score: 0.60
              precision    recall  f1-score   support

           0       0.75      0.50      0.60        12
           1       0.00      0.00      0.00         0
           2       0.45      0.66      0.53        32
           3       0.54      0.72      0.62        78
           4       0.58      0.58      0.58        65
           5       0.80      0.42      0.55        38
           6       0.74      0.67      0.71        52
           7       0.83      0.54      0.66        37
           8       0.70      0.47      0.57        91
           9       0.50      0.80      0.62        25
          10       0.53      1.00      0.70         8

    accuracy                           0.60       438
   macro avg       0.58      0.58      0.56       438
weighted avg       0.65      0.60      0.60       438



100%|██████████| 16/16 [00:55<00:00,  3.48s/it]



Fold 2 — F1 Score: 0.66
              precision    recall  f1-score   support

           0       1.00      0.73      0.85        15
           1       0.00      0.00      0.00         0
           2       0.92      0.64      0.75       124
           3       0.26      0.31      0.28        39
           4       0.91      0.59      0.72        49
           5       0.52      0.89      0.66        46
           6       0.67      0.82      0.73        22
           7       0.69      0.82      0.75        11
           8       0.45      0.52      0.48        50
           9       0.00      0.00      0.00         0
          10       1.00      0.80      0.89        20

    accuracy                           0.64       376
   macro avg       0.58      0.56      0.56       376
weighted avg       0.72      0.64      0.66       376



100%|██████████| 16/16 [00:58<00:00,  3.68s/it]



Fold 3 — F1 Score: 0.61
              precision    recall  f1-score   support

           0       0.95      0.83      0.88        23
           1       0.00      0.00      0.00        19
           2       0.44      0.94      0.60        16
           3       0.88      0.49      0.63       102
           4       0.56      0.78      0.65        65
           5       0.57      0.29      0.38        14
           6       0.35      0.50      0.41        14
           7       0.57      0.54      0.55        24
           8       0.50      0.44      0.47         9
           9       0.00      0.00      0.00         0
          10       1.00      1.00      1.00        21

    accuracy                           0.60       307
   macro avg       0.53      0.53      0.51       307
weighted avg       0.67      0.60      0.61       307



100%|██████████| 16/16 [01:00<00:00,  3.75s/it]



Fold 4 — F1 Score: 0.71
              precision    recall  f1-score   support

           0       1.00      0.89      0.94         9
           1       0.00      0.00      0.00         0
           2       0.90      0.80      0.85        79
           3       0.71      0.82      0.76        65
           4       0.62      0.72      0.67        39
           5       0.00      0.00      0.00         2
           6       0.62      0.62      0.62        26
           7       0.40      0.73      0.52        11
           8       0.54      0.68      0.60        19
           9       0.82      0.33      0.47        42
          10       0.92      0.92      0.92        12

    accuracy                           0.70       304
   macro avg       0.59      0.59      0.58       304
weighted avg       0.75      0.70      0.71       304



100%|██████████| 16/16 [01:01<00:00,  3.84s/it]


Fold 5 — F1 Score: 0.60
              precision    recall  f1-score   support

           0       0.64      0.93      0.76        15
           1       0.50      0.04      0.08        24
           2       0.67      0.47      0.55        17
           3       0.74      0.81      0.78        43
           4       0.46      0.74      0.57        34
           5       0.00      0.00      0.00         3
           6       0.10      0.40      0.16         5
           7       0.61      0.77      0.68        30
           8       0.75      0.43      0.55        28
           9       0.92      0.50      0.65        48
          10       1.00      1.00      1.00        16

    accuracy                           0.61       263
   macro avg       0.58      0.55      0.52       263
weighted avg       0.69      0.61      0.60       263


Average F1 Score over 5 folds: 0.64 (0.04)


In [12]:
import tabpfn
from tabpfn import load_fitted_tabpfn_model, save_fitted_tabpfn_model

# Train model on all data for deployment
X = gdf18[wavelength23_cols]
y = gdf18['class'].astype(int)

# Create a base TabPFN classifier
base_clf = TabPFNClassifier()
# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)
clf.fit(X, y)
tst = clf.predict(X[:5])

# -------------------------------------------------
# 2. Export the trained model to a file
# -------------------------------------------------

model_path = r"C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E1_2018_TABPFN.pkl"   # choose any path you like

joblib.dump(clf, model_path)

print(f"Model saved to {model_path}")

100%|██████████| 16/16 [01:04<00:00,  4.03s/it]

Model saved to C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E1_2018_TABPFN.pkl


### 2023

In [57]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import numpy as np

# Features and target
X = gdf23[wavelength23_cols]
y = gdf23['2023_class'].astype(int)
groups = gdf23['cluster']  # replace with the appropriate grouping column

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)

# GroupKFold setup
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Store metrics
f1_scores = []
accuracy_scores = []
reports = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X, y, groups)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # Accuracy
    acc = accuracy_score(y_test, y_pred)
    accuracy_scores.append(acc)

    # F1
    f1 = f1_score(y_test, y_pred, average='weighted')
    f1_scores.append(f1)
    
    # Report
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)
    
    print(f"\nFold {i + 1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy over {n_splits} folds: {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score over {n_splits} folds: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")



Fold 1 — Accuracy: 0.52, F1 Score: 0.52
              precision    recall  f1-score   support

           0       0.48      0.56      0.52        25
           1       0.00      0.00      0.00         0
           2       0.14      0.33      0.20        15
           3       0.45      0.62      0.53        64
           4       0.54      0.68      0.60        84
           5       0.73      0.24      0.36        34
           6       0.68      0.53      0.59        51
           7       0.66      0.59      0.62        46
           8       0.67      0.33      0.44        91
          10       0.42      1.00      0.59         8

    accuracy                           0.52       418
   macro avg       0.48      0.49      0.45       418
weighted avg       0.58      0.52      0.52       418


Fold 2 — Accuracy: 0.44, F1 Score: 0.45
              precision    recall  f1-score   support

           0       0.60      0.12      0.19        52
           1       0.00      0.00      0.00       

In [15]:
import joblib 

# Train model on all data for deployment
X = gdf23[wavelength23_cols]
y = gdf23['2023_class'].astype(int)

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)
clf.fit(X, y)

# -------------------------------------------------
# 2. Export the trained model to a file
# -------------------------------------------------

model_path = r"C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E1_2023_RF.pkl"   # choose any path you like
joblib.dump(clf, model_path)

print(f"Model saved to {model_path}")

Model saved to C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E1_2023_RF.pkl


In [58]:
from tabpfn import TabPFNClassifier
from tabpfn_extensions.many_class import ManyClassClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import numpy as np

# Features and target
X = gdf23[wavelength23_cols]
y = gdf23['2023_class'].astype(int)
groups = gdf23['cluster']  # ensure this column exists

# Create a base TabPFN classifier
base_clf = TabPFNClassifier()
# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)

# GroupKFold setup
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Store metrics
f1_scores = []
reports = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X, y, groups)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    f1 = f1_score(y_test, y_pred, average='weighted')
    f1_scores.append(f1)
    
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)
    
    print(f"\nFold {i + 1} — F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Print average F1
print(f"\nAverage F1 Score over {n_splits} folds: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")



Fold 1 — F1 Score: 0.58
              precision    recall  f1-score   support

           0       0.50      0.48      0.49        25
           1       0.00      0.00      0.00         0
           2       0.14      0.33      0.20        15
           3       0.51      0.61      0.56        64
           4       0.64      0.68      0.66        84
           5       0.95      0.62      0.75        34
           6       0.76      0.61      0.67        51
           7       0.69      0.76      0.72        46
           8       0.60      0.33      0.43        91
          10       0.40      1.00      0.57         8

    accuracy                           0.57       418
   macro avg       0.52      0.54      0.50       418
weighted avg       0.63      0.57      0.58       418


Fold 2 — F1 Score: 0.56
              precision    recall  f1-score   support

           0       0.53      0.15      0.24        52
           1       0.00      0.00      0.00         0
           2       0.76     

In [14]:
import tabpfn
from tabpfn import load_fitted_tabpfn_model, save_fitted_tabpfn_model

# Train model on all data for deployment
X = gdf23[wavelength23_cols]
y = gdf23['2023_class'].astype(int)

# Create a base TabPFN classifier
base_clf = TabPFNClassifier()
# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)
clf.fit(X, y)
# tst = clf.predict(X[:5])

# -------------------------------------------------
# 2. Export the trained model to a file
# -------------------------------------------------

model_path = r"C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E1_2023_TABPFN.pkl"   # choose any path you like

joblib.dump(clf, model_path)

print(f"Model saved to {model_path}")

Model saved to C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E1_2023_TABPFN.pkl


### Experiment 2.1: train on 2018 (all), predict on 2023 (all)

In [ ]:
# Add source labels
gdf18['source'] = '2018'
gdf23['source'] = '2023'

# Combine for consistent group splitting
combined = pd.concat([gdf18, gdf23], ignore_index=True)
X_all = combined[wavelength23_cols]
y_all = combined['class'].astype(int)
groups_all = combined['cluster']

# Track source to separate later
sources = combined['source']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Metrics
f1_scores = []
accuracy_scores = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = combined.iloc[train_idx]
    test_data = combined.iloc[test_idx]

    # subset 2018 train data to only include 2018 data
    train_data = train_data.loc[train_data['source'] == '2018']

    # subset 2023 test data to only include 2023 data
    test_data = test_data.loc[test_data['source'] == '2023']
    
    # Ensure all test groups come from df23subset only
    if not all(test_data['source'] == '2023'):
        raise ValueError("Test set contains data from 2018, which is not allowed.")

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy (only folds using 2023 test data): {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")



Fold 1 — Accuracy: 0.45, F1 Score: 0.44
              precision    recall  f1-score   support

           0       0.50      0.67      0.57        12
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00        17
           3       0.44      0.59      0.50        75
           4       0.39      0.71      0.50        65
           5       0.58      0.19      0.29        36
           6       0.71      0.38      0.50        52
           7       0.69      0.54      0.61        37
           8       0.49      0.40      0.44        91
           9       0.14      0.04      0.06        25
          10       0.73      1.00      0.84         8

    accuracy                           0.45       418
   macro avg       0.42      0.41      0.39       418
weighted avg       0.48      0.45      0.44       418


Fold 2 — Accuracy: 0.44, F1 Score: 0.43
              precision    recall  f1-score   support

           0       0.20      0.15      0.17       

In [16]:
# same model as experiment 1, 2018
import joblib 

# Train model on all data for deployment
X = gdf18[wavelength23_cols]
y = gdf18['class'].astype(int)

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)
clf.fit(X, y)

# -------------------------------------------------
# 2. Export the trained model to a file
# -------------------------------------------------

model_path = r"C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E21_2018_RF.pkl"   # choose any path you like
joblib.dump(clf, model_path)

print(f"Model saved to {model_path}")

Model saved to C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E21_2018_RF.pkl


In [ ]:
# Add source labels
gdf18['source'] = '2018'
gdf23['source'] = '2023'

# Combine for consistent group splitting
combined = pd.concat([gdf18, gdf23], ignore_index=True)
X_all = combined[wavelength23_cols]
y_all = combined['class'].astype(int)
groups_all = combined['cluster']

# Track source to separate later
sources = combined['source']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)

# Metrics
f1_scores = []
accuracy_scores = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = combined.iloc[train_idx]
    test_data = combined.iloc[test_idx]

    # subset 2018 train data to only include 2018 data
    train_data = train_data.loc[train_data['source'] == '2018']

    # subset 2023 test data to only include 2023 data
    test_data = test_data.loc[test_data['source'] == '2023']
    
    # Ensure all test groups come from df23subset only
    if not all(test_data['source'] == '2023'):
        raise ValueError("Test set contains data from 2018, which is not allowed.")

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy (only folds using 2023 test data): {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")


100%|██████████| 16/16 [00:53<00:00,  3.35s/it]



Fold 1 — Accuracy: 0.53, F1 Score: 0.52
              precision    recall  f1-score   support

           0       0.88      0.58      0.70        12
           1       0.00      0.00      0.00         0
           2       0.22      0.24      0.23        17
           3       0.54      0.48      0.51        75
           4       0.43      0.85      0.57        65
           5       1.00      0.17      0.29        36
           6       0.84      0.60      0.70        52
           7       0.62      0.70      0.66        37
           8       0.52      0.52      0.52        91
           9       0.29      0.08      0.12        25
          10       0.80      1.00      0.89         8

    accuracy                           0.53       418
   macro avg       0.56      0.47      0.47       418
weighted avg       0.59      0.53      0.52       418



100%|██████████| 16/16 [00:56<00:00,  3.55s/it]



Fold 2 — Accuracy: 0.58, F1 Score: 0.59
              precision    recall  f1-score   support

           0       0.25      0.15      0.19        13
           1       0.00      0.00      0.00         0
           2       0.84      0.41      0.55       102
           3       0.31      0.47      0.37        38
           4       0.55      0.73      0.63        49
           5       0.88      0.50      0.64        46
           6       0.72      0.82      0.77        22
           7       0.41      0.64      0.50        11
           8       0.63      0.86      0.73        50
           9       0.00      0.00      0.00         0
          10       0.67      0.80      0.73        20

    accuracy                           0.58       351
   macro avg       0.48      0.49      0.46       351
weighted avg       0.66      0.58      0.59       351



100%|██████████| 16/16 [01:00<00:00,  3.78s/it]



Fold 3 — Accuracy: 0.53, F1 Score: 0.52
              precision    recall  f1-score   support

           0       0.54      0.65      0.59        23
           1       1.00      0.05      0.10        19
           2       0.30      0.62      0.41        16
           3       0.93      0.26      0.41        99
           4       0.51      0.85      0.64        65
           5       0.78      0.50      0.61        14
           6       0.28      0.36      0.31        14
           7       1.00      0.67      0.80        24
           8       0.12      0.44      0.19         9
           9       0.00      0.00      0.00         0
          10       1.00      1.00      1.00        21

    accuracy                           0.53       304
   macro avg       0.59      0.49      0.46       304
weighted avg       0.73      0.53      0.52       304



100%|██████████| 16/16 [01:03<00:00,  3.98s/it]



Fold 4 — Accuracy: 0.49, F1 Score: 0.48
              precision    recall  f1-score   support

           0       0.80      0.57      0.67         7
           1       0.00      0.00      0.00         0
           2       0.97      0.41      0.58        73
           3       0.65      0.55      0.60        62
           4       0.38      0.79      0.51        39
           5       0.00      0.00      0.00         2
           6       0.59      0.38      0.47        26
           7       0.50      0.82      0.62        11
           8       0.21      0.74      0.33        19
           9       0.00      0.00      0.00        42
          10       1.00      1.00      1.00        12

    accuracy                           0.49       293
   macro avg       0.46      0.48      0.43       293
weighted avg       0.57      0.49      0.48       293



100%|██████████| 16/16 [01:05<00:00,  4.12s/it]


Fold 5 — Accuracy: 0.47, F1 Score: 0.40
              precision    recall  f1-score   support

           0       0.82      0.60      0.69        15
           1       0.00      0.00      0.00        22
           2       0.46      0.35      0.40        17
           3       0.74      0.59      0.66        39
           4       0.34      0.59      0.43        34
           5       0.00      0.00      0.00         3
           6       0.20      0.20      0.20         5
           7       0.45      0.83      0.58        30
           8       0.34      0.75      0.47        28
           9       0.00      0.00      0.00        48
          10       0.80      1.00      0.89        16

    accuracy                           0.47       257
   macro avg       0.38      0.45      0.39       257
weighted avg       0.38      0.47      0.40       257


Average Accuracy (only folds using 2023 test data): 0.52 (0.04)
Average F1 Score: 0.50 (0.06)


In [17]:
import tabpfn
from tabpfn import load_fitted_tabpfn_model, save_fitted_tabpfn_model

# Train model on all data for deployment
X = gdf18[wavelength23_cols]
y = gdf18['class'].astype(int)

# Create a base TabPFN classifier
base_clf = TabPFNClassifier()
# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)
clf.fit(X, y)
# tst = clf.predict(X[:5])

# -------------------------------------------------
# 2. Export the trained model to a file
# -------------------------------------------------

model_path = r"C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E21_2018_TABPFN.pkl"   # choose any path you like

joblib.dump(clf, model_path)

print(f"Model saved to {model_path}")

Model saved to C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E21_2018_TABPFN.pkl


### Experiment 2.2: with L2 normalization

In [12]:
def l2_normalize_dataframe(df, wavelength_cols):
    # Compute L2 norm per row
    l2_norm = np.sqrt((df[wavelength_cols] ** 2).sum(axis=1))
    
    # Avoid division by zero
    l2_norm = l2_norm.replace(0, np.nan)
    
    # Normalize the wavelength columns
    df[wavelength_cols] = df[wavelength_cols].div(l2_norm, axis=0)
    
    return df

In [13]:
# Add source labels
gdf18['source'] = '2018'
gdf23['source'] = '2023'

# Combine for consistent group splitting
combined = pd.concat([gdf18, gdf23], ignore_index=True)
combinednorm = l2_normalize_dataframe(combined, wavelength23_cols)
X_all = combinednorm[wavelength23_cols]
y_all = combinednorm['class'].astype(int)
groups_all = combinednorm['cluster']

# Track source to separate later
sources = combinednorm['source']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Metrics
f1_scores = []
accuracy_scores = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = combinednorm.iloc[train_idx]
    test_data = combinednorm.iloc[test_idx]

    # subset 2018 train data to only include 2018 data
    train_data = train_data.loc[train_data['source'] == '2018']

    # subset 2023 test data to only include 2023 data
    test_data = test_data.loc[test_data['source'] == '2023']
    
    # Ensure all test groups come from df23subset only
    if not all(test_data['source'] == '2023'):
        raise ValueError("Test set contains data from 2018, which is not allowed.")

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy (only folds using 2023 test data): {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")



Fold 1 — Accuracy: 0.47, F1 Score: 0.46
              precision    recall  f1-score   support

           0       0.78      0.58      0.67        12
           1       0.00      0.00      0.00         0
           2       0.26      0.29      0.28        17
           3       0.44      0.55      0.49        75
           4       0.42      0.77      0.54        65
           5       0.82      0.25      0.38        36
           6       0.79      0.29      0.42        52
           7       0.66      0.57      0.61        37
           8       0.46      0.44      0.45        91
           9       0.17      0.04      0.06        25
          10       0.80      1.00      0.89         8

    accuracy                           0.47       418
   macro avg       0.51      0.43      0.44       418
weighted avg       0.53      0.47      0.46       418


Fold 2 — Accuracy: 0.46, F1 Score: 0.46
              precision    recall  f1-score   support

           0       0.17      0.15      0.16       

In [ ]:
import joblib 

# Train model on all data for deployment
combinednorm = l2_normalize_dataframe(gdf18, wavelength23_cols)
X = combinednorm[wavelength23_cols]
y = combinednorm['class'].astype(int)

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)
clf.fit(X, y)

# -------------------------------------------------
# 2. Export the trained model to a file
# -------------------------------------------------

model_path = r"C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E22_2018_RF.pkl"   # choose any path you like
joblib.dump(clf, model_path)

print(f"Model saved to {model_path}")

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1]
Model saved to C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E22_2018_RF.pkl


In [ ]:
# Add source labels
gdf18['source'] = '2018'
gdf23['source'] = '2023'

# Combine for consistent group splitting
combined = pd.concat([gdf18, gdf23], ignore_index=True)
combinednorm = l2_normalize_dataframe(combined, wavelength23_cols)
X_all = combinednorm[wavelength23_cols]
y_all = combinednorm['class'].astype(int)
groups_all = combinednorm['cluster']

# Track source to separate later
sources = combined['source']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)

# Metrics
f1_scores = []
accuracy_scores = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = combinednorm.iloc[train_idx]
    test_data = combinednorm.iloc[test_idx]

    # subset 2018 train data to only include 2018 data
    train_data = train_data.loc[train_data['source'] == '2018']

    # subset 2023 test data to only include 2023 data
    test_data = test_data.loc[test_data['source'] == '2023']
    
    # Ensure all test groups come from df23subset only
    if not all(test_data['source'] == '2023'):
        raise ValueError("Test set contains data from 2018, which is not allowed.")

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy (only folds using 2023 test data): {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")


100%|██████████| 16/16 [00:53<00:00,  3.37s/it]



Fold 1 — Accuracy: 0.49, F1 Score: 0.47
              precision    recall  f1-score   support

           0       1.00      0.50      0.67        12
           1       0.00      0.00      0.00         0
           2       0.25      0.18      0.21        17
           3       0.47      0.39      0.42        75
           4       0.45      0.83      0.59        65
           5       1.00      0.19      0.33        36
           6       0.74      0.38      0.51        52
           7       0.64      0.62      0.63        37
           8       0.42      0.57      0.49        91
           9       0.29      0.08      0.12        25
          10       0.73      1.00      0.84         8

    accuracy                           0.49       418
   macro avg       0.54      0.43      0.44       418
weighted avg       0.55      0.49      0.47       418



100%|██████████| 16/16 [00:56<00:00,  3.54s/it]



Fold 2 — Accuracy: 0.57, F1 Score: 0.56
              precision    recall  f1-score   support

           0       0.30      0.23      0.26        13
           1       0.00      0.00      0.00         0
           2       0.84      0.26      0.40       102
           3       0.31      0.50      0.38        38
           4       0.53      0.76      0.62        49
           5       0.74      0.57      0.64        46
           6       0.59      0.86      0.70        22
           7       0.91      0.91      0.91        11
           8       0.65      0.86      0.74        50
           9       0.00      0.00      0.00         0
          10       0.67      0.80      0.73        20

    accuracy                           0.57       351
   macro avg       0.50      0.52      0.49       351
weighted avg       0.66      0.57      0.56       351



100%|██████████| 16/16 [01:01<00:00,  3.84s/it]



Fold 3 — Accuracy: 0.48, F1 Score: 0.50
              precision    recall  f1-score   support

           0       0.40      0.70      0.51        23
           1       1.00      0.21      0.35        19
           2       0.35      0.56      0.43        16
           3       0.96      0.22      0.36        99
           4       0.48      0.71      0.57        65
           5       1.00      0.43      0.60        14
           6       0.32      0.43      0.36        14
           7       0.93      0.58      0.72        24
           8       0.12      0.56      0.20         9
           9       0.00      0.00      0.00         0
          10       1.00      0.90      0.95        21

    accuracy                           0.48       304
   macro avg       0.60      0.48      0.46       304
weighted avg       0.73      0.48      0.50       304



100%|██████████| 16/16 [01:04<00:00,  4.01s/it]



Fold 4 — Accuracy: 0.44, F1 Score: 0.44
              precision    recall  f1-score   support

           0       0.80      0.57      0.67         7
           1       0.00      0.00      0.00         0
           2       0.90      0.36      0.51        73
           3       0.69      0.47      0.56        62
           4       0.37      0.79      0.50        39
           5       0.00      0.00      0.00         2
           6       0.44      0.31      0.36        26
           7       0.39      0.64      0.48        11
           8       0.18      0.68      0.28        19
           9       0.00      0.00      0.00        42
          10       1.00      1.00      1.00        12

    accuracy                           0.44       293
   macro avg       0.43      0.44      0.40       293
weighted avg       0.54      0.44      0.44       293



100%|██████████| 16/16 [01:06<00:00,  4.17s/it]


Fold 5 — Accuracy: 0.48, F1 Score: 0.43
              precision    recall  f1-score   support

           0       0.69      0.60      0.64        15
           1       0.00      0.00      0.00        22
           2       0.50      0.29      0.37        17
           3       0.71      0.62      0.66        39
           4       0.34      0.59      0.43        34
           5       0.33      0.33      0.33         3
           6       0.12      0.20      0.15         5
           7       0.48      0.80      0.60        30
           8       0.38      0.75      0.50        28
           9       0.60      0.06      0.11        48
          10       0.80      1.00      0.89        16

    accuracy                           0.48       257
   macro avg       0.45      0.48      0.43       257
weighted avg       0.49      0.48      0.43       257


Average Accuracy (only folds using 2023 test data): 0.49 (0.04)
Average F1 Score: 0.48 (0.05)


In [20]:
# Train model on all data for deployment
combinednorm = l2_normalize_dataframe(gdf18, wavelength23_cols)
X = combinednorm[wavelength23_cols]
y = combinednorm['class'].astype(int)

# Create a base TabPFN classifier
base_clf = TabPFNClassifier()
# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)
clf.fit(X, y)
# tst = clf.predict(X[:5])

# -------------------------------------------------
# 2. Export the trained model to a file
# -------------------------------------------------

model_path = r"C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E22_2018_TABPFN.pkl"   # choose any path you like

joblib.dump(clf, model_path)

print(f"Model saved to {model_path}")

Model saved to C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E22_2018_TABPFN.pkl


### Experiment 3: Wessels 2016 using IRMAD

refer to notebook IRMAD experiments.ipynb

### Experiment 4.1: E2.4+filtering

In [21]:
def l2_normalize_dataframe(df, wavelength_cols):
    # Compute L2 norm per row
    l2_norm = np.sqrt((df[wavelength_cols] ** 2).sum(axis=1))
    
    # Avoid division by zero
    l2_norm = l2_norm.replace(0, np.nan)
    
    # Normalize the wavelength columns
    df[wavelength_cols] = df[wavelength_cols].div(l2_norm, axis=0)
    
    return df

In [22]:
wavelength23_cols = ['B2','B3', 'B4', 'B5', 'B6', 'B7', 'B8','B11','B12']

In [ ]:
# Add source labels
gdf18['source'] = '2018'
gdf23['source'] = '2023'

# Combine for consistent group splitting
combined = pd.concat([gdf18, gdf23.loc[gdf23['change']==0]], ignore_index=True)
combinednorm = l2_normalize_dataframe(combined, wavelength23_cols)
X_all = combinednorm[wavelength23_cols]
y_all = combinednorm['class'].astype(int)
groups_all = combinednorm['cluster']

# Track source to separate later
sources = combinednorm['source']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Metrics
f1_scores = []
accuracy_scores = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = combinednorm.iloc[train_idx]
    test_data = combinednorm.iloc[test_idx]

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy (only folds using 2023 test data): {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")



Fold 1 — Accuracy: 0.51, F1 Score: 0.51
              precision    recall  f1-score   support

           0       0.83      0.42      0.56        24
           1       0.00      0.00      0.00         0
           2       0.25      0.55      0.34        47
           3       0.45      0.73      0.56       142
           4       0.54      0.67      0.60       123
           5       0.83      0.21      0.33        72
           6       0.75      0.47      0.57       103
           7       0.68      0.53      0.60        74
           8       0.60      0.32      0.42       181
           9       0.58      0.60      0.59        25
          10       0.50      1.00      0.67        16

    accuracy                           0.51       807
   macro avg       0.55      0.50      0.48       807
weighted avg       0.60      0.51      0.51       807


Fold 2 — Accuracy: 0.55, F1 Score: 0.57
              precision    recall  f1-score   support

           0       0.64      0.35      0.45       

In [23]:
import joblib 

# Train model on all data for deployment
combined = pd.concat([gdf18, gdf23.loc[gdf23['change']==0]], ignore_index=True)
combinednorm = l2_normalize_dataframe(combined, wavelength23_cols)
X = combinednorm[wavelength23_cols]
y = combinednorm['class'].astype(int)

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)
clf.fit(X, y)

# -------------------------------------------------
# 2. Export the trained model to a file
# -------------------------------------------------

model_path = r"C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E41_2018_RF.pkl"   # choose any path you like
joblib.dump(clf, model_path)

print(f"Model saved to {model_path}")

Model saved to C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E41_2018_RF.pkl


In [ ]:
# Add source labels
gdf18['source'] = '2018'
gdf23['source'] = '2023'

# Combine for consistent group splitting
combined = pd.concat([gdf18, gdf23.loc[gdf23['change']==0]], ignore_index=True)
combinednorm = l2_normalize_dataframe(combined, wavelength23_cols)
X_all = combinednorm[wavelength23_cols]
y_all = combinednorm['class'].astype(int)
groups_all = combinednorm['cluster']

# Track source to separate later
sources = combinednorm['source']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)

# Metrics
f1_scores = []
accuracy_scores = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = combinednorm.iloc[train_idx]
    test_data = combinednorm.iloc[test_idx]

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy (only folds using 2023 test data): {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")


100%|██████████| 16/16 [02:01<00:00,  7.61s/it]



Fold 1 — Accuracy: 0.59, F1 Score: 0.59
              precision    recall  f1-score   support

           0       0.69      0.38      0.49        24
           1       0.00      0.00      0.00         0
           2       0.36      0.66      0.47        47
           3       0.53      0.73      0.61       142
           4       0.58      0.65      0.62       123
           5       0.81      0.35      0.49        72
           6       0.81      0.66      0.73       103
           7       0.70      0.61      0.65        74
           8       0.66      0.45      0.54       181
           9       0.61      0.68      0.64        25
          10       0.52      1.00      0.68        16

    accuracy                           0.59       807
   macro avg       0.57      0.56      0.54       807
weighted avg       0.64      0.59      0.59       807



100%|██████████| 16/16 [02:13<00:00,  8.32s/it]



Fold 2 — Accuracy: 0.67, F1 Score: 0.69
              precision    recall  f1-score   support

           0       0.78      0.70      0.74        20
           1       0.00      0.00      0.00         0
           2       0.90      0.66      0.76       192
           3       0.37      0.64      0.47        70
           4       0.73      0.56      0.63        81
           5       0.71      0.75      0.73        91
           6       0.60      0.77      0.67        44
           7       0.95      0.86      0.90        22
           8       0.59      0.56      0.57        97
           9       0.00      0.00      0.00         0
          10       1.00      0.85      0.92        39

    accuracy                           0.67       656
   macro avg       0.60      0.58      0.58       656
weighted avg       0.73      0.67      0.69       656



100%|██████████| 16/16 [02:17<00:00,  8.60s/it]



Fold 3 — Accuracy: 0.56, F1 Score: 0.56
              precision    recall  f1-score   support

           0       0.68      0.85      0.76        46
           1       0.80      0.11      0.19        38
           2       0.27      0.66      0.39        32
           3       0.90      0.40      0.56       201
           4       0.53      0.70      0.60       125
           5       0.52      0.61      0.56        28
           6       0.34      0.39      0.37        28
           7       0.48      0.58      0.53        48
           8       0.27      0.50      0.35        18
           9       0.00      0.00      0.00         0
          10       1.00      0.98      0.99        41

    accuracy                           0.56       605
   macro avg       0.53      0.52      0.48       605
weighted avg       0.68      0.56      0.56       605



100%|██████████| 16/16 [02:18<00:00,  8.65s/it]



Fold 4 — Accuracy: 0.67, F1 Score: 0.69
              precision    recall  f1-score   support

           0       0.86      0.75      0.80        16
           1       0.00      0.00      0.00         0
           2       0.89      0.63      0.74       148
           3       0.74      0.80      0.77       126
           4       0.66      0.75      0.70        76
           5       0.06      0.25      0.09         4
           6       0.55      0.52      0.53        52
           7       0.44      0.64      0.52        22
           8       0.42      0.63      0.51        38
           9       0.94      0.36      0.52        42
          10       1.00      0.96      0.98        24

    accuracy                           0.67       548
   macro avg       0.60      0.57      0.56       548
weighted avg       0.74      0.67      0.69       548



100%|██████████| 16/16 [02:17<00:00,  8.60s/it]


Fold 5 — Accuracy: 0.60, F1 Score: 0.59
              precision    recall  f1-score   support

           0       0.79      0.79      0.79        28
           1       0.00      0.00      0.00        45
           2       0.43      0.47      0.45        34
           3       0.72      0.74      0.73        82
           4       0.56      0.72      0.63        65
           5       0.00      0.00      0.00         6
           6       0.09      0.20      0.12        10
           7       0.65      0.72      0.68        60
           8       0.52      0.55      0.53        56
           9       0.87      0.54      0.67        48
          10       0.94      1.00      0.97        32

    accuracy                           0.60       466
   macro avg       0.51      0.52      0.51       466
weighted avg       0.58      0.60      0.59       466


Average Accuracy (only folds using 2023 test data): 0.62 (0.04)
Average F1 Score: 0.62 (0.05)


In [24]:
# Train model on all data for deployment
combined = pd.concat([gdf18, gdf23.loc[gdf23['change']==0]], ignore_index=True)
combinednorm = l2_normalize_dataframe(combined, wavelength23_cols)
X = combinednorm[wavelength23_cols]
y = combinednorm['class'].astype(int)

# Create a base TabPFN classifier
base_clf = TabPFNClassifier()
# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)
clf.fit(X, y)
# tst = clf.predict(X[:5])

# -------------------------------------------------
# 2. Export the trained model to a file
# -------------------------------------------------

model_path = r"C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E41_2018_TABPFN.pkl"   # choose any path you like

joblib.dump(clf, model_path)

print(f"Model saved to {model_path}")

Model saved to C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E41_2018_TABPFN.pkl


### Experiment 4.2: E2.4+filtering

In [12]:
def l2_normalize_dataframe(df, wavelength_cols):
    # Compute L2 norm per row
    l2_norm = np.sqrt((df[wavelength_cols] ** 2).sum(axis=1))
    
    # Avoid division by zero
    l2_norm = l2_norm.replace(0, np.nan)
    
    # Normalize the wavelength columns
    df[wavelength_cols] = df[wavelength_cols].div(l2_norm, axis=0)
    
    return df

In [25]:
wavelength23_cols = ['B2','B3', 'B4', 'B5', 'B6', 'B7', 'B8','B11','B12']

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import pandas as pd
import numpy as np

# Add source labels
gdf18['source'] = '2018'
gdf23['source'] = '2023'

# Combine for consistent group splitting
combined = pd.concat([gdf18, gdf23.loc[gdf23['imad_change']==1]], ignore_index=True)
combinednorm = l2_normalize_dataframe(combined, wavelength23_cols)
X_all = combinednorm[wavelength23_cols]
y_all = combinednorm['class'].astype(int)
groups_all = combinednorm['cluster']

# Track source to separate later
sources = combinednorm['source']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Metrics
f1_scores = []
accuracy_scores = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = combinednorm.iloc[train_idx]
    test_data = combinednorm.iloc[test_idx]

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy (only folds using 2023 test data): {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")



Fold 1 — Accuracy: 0.49, F1 Score: 0.48
              precision    recall  f1-score   support

           0       0.75      0.23      0.35        13
           1       0.00      0.00      0.00         0
           2       0.28      0.54      0.37        39
           3       0.49      0.76      0.59       125
           4       0.51      0.62      0.56        88
           5       0.86      0.26      0.39        47
           6       0.73      0.33      0.45        73
           7       0.56      0.38      0.45        50
           8       0.56      0.28      0.37       124
           9       0.54      0.71      0.61        31
          10       0.50      1.00      0.67         8

    accuracy                           0.49       598
   macro avg       0.52      0.46      0.44       598
weighted avg       0.56      0.49      0.48       598


Fold 2 — Accuracy: 0.56, F1 Score: 0.57
              precision    recall  f1-score   support

           0       1.00      0.12      0.22       

In [25]:
import joblib 

# Train model on all data for deployment
combined = pd.concat([gdf18, gdf23.loc[gdf23['imad_change']==1]], ignore_index=True)
combinednorm = l2_normalize_dataframe(combined, wavelength23_cols)
X = combinednorm[wavelength23_cols]
y = combinednorm['class'].astype(int)

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)
clf.fit(X, y)

# -------------------------------------------------
# 2. Export the trained model to a file
# -------------------------------------------------

model_path = r"C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E42_2018_RF.pkl"   # choose any path you like
joblib.dump(clf, model_path)

print(f"Model saved to {model_path}")

Model saved to C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E42_2018_RF.pkl


In [27]:
# Add source labels
gdf18['source'] = '2018'
gdf23['source'] = '2023'

# Combine for consistent group splitting
combined = pd.concat([gdf18, gdf23.loc[gdf23['imad_change']==1]], ignore_index=True)
combinednorm = l2_normalize_dataframe(combined, wavelength23_cols)
X_all = combinednorm[wavelength23_cols]
y_all = combinednorm['class'].astype(int)
groups_all = combinednorm['cluster']

# Track source to separate later
sources = combinednorm['source']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)

# Metrics
f1_scores = []
accuracy_scores = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = combinednorm.iloc[train_idx]
    test_data = combinednorm.iloc[test_idx]

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy (only folds using 2023 test data): {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")


100%|██████████| 16/16 [01:28<00:00,  5.52s/it]



Fold 1 — Accuracy: 0.58, F1 Score: 0.58
              precision    recall  f1-score   support

           0       0.80      0.31      0.44        13
           1       0.00      0.00      0.00         0
           2       0.42      0.67      0.51        39
           3       0.54      0.74      0.63       125
           4       0.54      0.56      0.55        88
           5       0.77      0.36      0.49        47
           6       0.80      0.64      0.71        73
           7       0.69      0.50      0.58        50
           8       0.63      0.46      0.53       124
           9       0.57      0.74      0.65        31
          10       0.47      1.00      0.64         8

    accuracy                           0.58       598
   macro avg       0.57      0.54      0.52       598
weighted avg       0.62      0.58      0.58       598



100%|██████████| 16/16 [01:34<00:00,  5.89s/it]



Fold 2 — Accuracy: 0.68, F1 Score: 0.70
              precision    recall  f1-score   support

           0       1.00      0.62      0.77        16
           1       0.00      0.00      0.00         0
           2       0.88      0.78      0.83       147
           3       0.40      0.68      0.50        60
           4       0.86      0.57      0.69        77
           5       0.69      0.81      0.74        59
           6       0.57      0.79      0.67        29
           7       0.94      0.83      0.88        18
           8       0.52      0.39      0.45        74
           9       0.00      0.00      0.00         0
          10       1.00      0.81      0.89        21

    accuracy                           0.68       501
   macro avg       0.62      0.57      0.58       501
weighted avg       0.74      0.68      0.70       501



100%|██████████| 16/16 [01:39<00:00,  6.20s/it]



Fold 3 — Accuracy: 0.54, F1 Score: 0.56
              precision    recall  f1-score   support

           0       0.57      0.83      0.68        24
           1       1.00      0.13      0.23        23
           2       0.31      0.65      0.42        23
           3       0.89      0.43      0.58       185
           4       0.51      0.64      0.57       116
           5       0.71      0.48      0.57        21
           6       0.40      0.50      0.44        20
           7       0.47      0.61      0.53        36
           8       0.33      0.58      0.42        12
           9       0.00      0.00      0.00         0
          10       1.00      1.00      1.00        21

    accuracy                           0.54       481
   macro avg       0.56      0.53      0.50       481
weighted avg       0.69      0.54      0.56       481



100%|██████████| 16/16 [01:43<00:00,  6.44s/it]



Fold 4 — Accuracy: 0.67, F1 Score: 0.67
              precision    recall  f1-score   support

           0       1.00      0.78      0.88         9
           1       0.00      0.00      0.00         0
           2       0.90      0.72      0.80       122
           3       0.73      0.84      0.78       107
           4       0.52      0.71      0.60        63
           5       0.00      0.00      0.00         2
           6       0.47      0.45      0.46        31
           7       0.43      0.63      0.51        19
           8       0.47      0.65      0.54        31
           9       0.79      0.29      0.42        66
          10       1.00      0.94      0.97        16

    accuracy                           0.67       466
   macro avg       0.57      0.55      0.54       466
weighted avg       0.72      0.67      0.67       466



100%|██████████| 16/16 [01:46<00:00,  6.65s/it]


Fold 5 — Accuracy: 0.61, F1 Score: 0.60
              precision    recall  f1-score   support

           0       0.71      0.75      0.73        16
           1       0.83      0.20      0.32        25
           2       0.50      0.40      0.44        30
           3       0.71      0.80      0.75        66
           4       0.44      0.68      0.54        57
           5       0.00      0.00      0.00         4
           6       0.18      0.38      0.24         8
           7       0.62      0.73      0.67        52
           8       0.61      0.53      0.56        38
           9       0.76      0.43      0.55        67
          10       1.00      1.00      1.00        27

    accuracy                           0.61       390
   macro avg       0.58      0.54      0.53       390
weighted avg       0.65      0.61      0.60       390


Average Accuracy (only folds using 2023 test data): 0.62 (0.05)
Average F1 Score: 0.62 (0.05)


In [26]:
# Train model on all data for deployment
combined = pd.concat([gdf18, gdf23.loc[gdf23['imad_change']==1]], ignore_index=True)
combinednorm = l2_normalize_dataframe(combined, wavelength23_cols)
X = combinednorm[wavelength23_cols]
y = combinednorm['class'].astype(int)

# Create a base TabPFN classifier
base_clf = TabPFNClassifier()
# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)
clf.fit(X, y)
# tst = clf.predict(X[:5])

# -------------------------------------------------
# 2. Export the trained model to a file
# -------------------------------------------------

model_path = r"C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E42_2018_TABPFN.pkl"   # choose any path you like

joblib.dump(clf, model_path)

print(f"Model saved to {model_path}")

Model saved to C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E42_2018_TABPFN.pkl


### Experiment 5.1: SSL

In [88]:
# with spatial CV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import pandas as pd
import numpy as np

# Add source labels
gdf18['source'] = '2018'
gdf23['source'] = '2023'

# Combine for consistent group splitting
combined = pd.concat([gdf18, gdf23.loc[gdf23['change']==0]], ignore_index=True)
X_all = combined[wavelength23_cols]
y_all = combined['class'].astype(int)
groups_all = combined['cluster']

# Track source to separate later
sources = combined['source']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Metrics
f1_scores = []
accuracy_scores = []

# Collect all 2023 change subset predictions
df23_all_preds = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = combined.iloc[train_idx]
    test_data = combined.iloc[test_idx]

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)
    print(i , groups_all.iloc[test_idx].unique())

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

    # Get pseudo labels for 2023 change points
    df23changesubset = gdf23[(gdf23['change'] == 1) & (gdf23['cluster']==groups_all.iloc[test_idx].unique()[0])].copy()
    X = df23changesubset[wavelength23_cols]
    predicted_labels = clf.predict(X)
    df23changesubset['predicted_class'] = predicted_labels
    df23_all_preds.append(df23changesubset)
    print(df23changesubset['predicted_class'].value_counts())

df23_all_preds = pd.concat(df23_all_preds, ignore_index=False)

# Summary
print(f"\nAverage Accuracy (only folds using 2023 test data): {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")


0 [3]

Fold 1 — Accuracy: 0.48, F1 Score: 0.48
              precision    recall  f1-score   support

           0       0.55      0.46      0.50        24
           1       0.00      0.00      0.00         0
           2       0.22      0.43      0.29        47
           3       0.42      0.74      0.53       142
           4       0.48      0.61      0.54       123
           5       0.73      0.26      0.39        72
           6       0.79      0.43      0.55       103
           7       0.73      0.51      0.60        74
           8       0.64      0.27      0.38       181
           9       0.56      0.56      0.56        25
          10       0.50      1.00      0.67        16

    accuracy                           0.48       807
   macro avg       0.51      0.48      0.46       807
weighted avg       0.58      0.48      0.48       807

predicted_class
4    27
7     7
3     7
8     3
6     1
9     1
2     1
0     1
1     1
Name: count, dtype: int64
1 [0]

Fold 2 — Accuracy: 

In [89]:
df23_all_preds['class'] = df23_all_preds['predicted_class'].astype(int)
dfallsemisuperivised = pd.concat([gdf18[['class', 'cluster']+wavelength23_cols], gdf23.loc[gdf23['change']==0][['class', 'cluster']+wavelength23_cols], df23_all_preds[['class', 'cluster']+wavelength23_cols]], ignore_index=True)

In [90]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import pandas as pd
import numpy as np

# Combine for consistent group splitting
X_all = dfallsemisuperivised[wavelength23_cols]
y_all = dfallsemisuperivised['class'].astype(int)
groups_all = dfallsemisuperivised['cluster']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Metrics
f1_scores = []
accuracy_scores = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = dfallsemisuperivised.iloc[train_idx]
    test_data = dfallsemisuperivised.iloc[test_idx]

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy: {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")


Fold 1 — Accuracy: 0.53, F1 Score: 0.53
              precision    recall  f1-score   support

           0       0.61      0.44      0.51        25
           1       0.06      1.00      0.12         1
           2       0.23      0.44      0.30        48
           3       0.45      0.73      0.56       149
           4       0.57      0.73      0.64       150
           5       0.79      0.26      0.40        72
           6       0.80      0.49      0.61       104
           7       0.72      0.58      0.64        81
           8       0.65      0.30      0.41       184
           9       0.63      0.65      0.64        26
          10       0.50      1.00      0.67        16

    accuracy                           0.53       856
   macro avg       0.55      0.60      0.50       856
weighted avg       0.61      0.53      0.53       856


Fold 2 — Accuracy: 0.56, F1 Score: 0.58
              precision    recall  f1-score   support

           0       0.82      0.41      0.55       

In [13]:
import joblib 

# Train model on all data for deployment
combined = pd.concat([gdf18, gdf23.loc[gdf23['change']==0]], ignore_index=True)
combinednorm = l2_normalize_dataframe(combined, wavelength23_cols)
X = combinednorm[wavelength23_cols]
y = combinednorm['class'].astype(int)

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)
clf.fit(X, y)

# Get pseudo labels for 2023 change points
df23changesubset = gdf23[(gdf23['change'] == 1)].copy()
X = df23changesubset[wavelength23_cols]
predicted_labels = clf.predict(X)
df23changesubset['predicted_class'] = predicted_labels

df23changesubset['class'] = df23changesubset['predicted_class'].astype(int)
dfallsemisuperivised = pd.concat([gdf18[['class', 'cluster']+wavelength23_cols], gdf23.loc[gdf23['change']==0][['class', 'cluster']+wavelength23_cols], df23changesubset[['class', 'cluster']+wavelength23_cols]], ignore_index=True)
X = dfallsemisuperivised[wavelength23_cols]
y = dfallsemisuperivised['class'].astype(int)
sslclf = RandomForestClassifier(random_state=42, n_jobs=-1)
sslclf.fit(X, y)
# -------------------------------------------------
# 2. Export the trained model to a file
# -------------------------------------------------

model_path = r"C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E51_2018_RF.pkl"   # choose any path you like
joblib.dump(sslclf, model_path)

print(f"Model saved to {model_path}")

Model saved to C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E51_2018_RF.pkl


TABPFN

In [91]:
# with spatial CV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import pandas as pd
import numpy as np

# Add source labels
gdf18['source'] = '2018'
gdf23['source'] = '2023'

# Combine for consistent group splitting
combined = pd.concat([gdf18, gdf23.loc[gdf23['change']==0]], ignore_index=True)
X_all = combined[wavelength23_cols]
y_all = combined['class'].astype(int)
groups_all = combined['cluster']

# Track source to separate later
sources = combined['source']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)

# Metrics
f1_scores = []
accuracy_scores = []

# Collect all 2023 change subset predictions
df23_all_preds = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = combined.iloc[train_idx]
    test_data = combined.iloc[test_idx]

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)
    print(i , groups_all.iloc[test_idx].unique())

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

    # Get pseudo labels for 2023 change points
    df23changesubset = gdf23[(gdf23['change'] == 1) & (gdf23['cluster']==groups_all.iloc[test_idx].unique()[0])].copy()
    X = df23changesubset[wavelength23_cols]
    predicted_labels = clf.predict(X)
    df23changesubset['predicted_class'] = predicted_labels
    df23_all_preds.append(df23changesubset)
    print(df23changesubset['predicted_class'].value_counts())

df23_all_preds = pd.concat(df23_all_preds, ignore_index=False)

# Summary
print(f"\nAverage Accuracy (only folds using 2023 test data): {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")


0 [3]


100%|██████████| 16/16 [02:03<00:00,  7.73s/it]



Fold 1 — Accuracy: 0.61, F1 Score: 0.61
              precision    recall  f1-score   support

           0       0.76      0.54      0.63        24
           1       0.00      0.00      0.00         0
           2       0.34      0.62      0.44        47
           3       0.56      0.74      0.63       142
           4       0.56      0.71      0.63       123
           5       0.81      0.40      0.54        72
           6       0.83      0.69      0.75       103
           7       0.76      0.61      0.68        74
           8       0.67      0.41      0.51       181
           9       0.66      0.76      0.70        25
          10       0.62      1.00      0.76        16

    accuracy                           0.61       807
   macro avg       0.60      0.59      0.57       807
weighted avg       0.66      0.61      0.61       807



100%|██████████| 16/16 [01:40<00:00,  6.28s/it]


predicted_class
4    24
7    11
3     6
8     4
1     2
9     1
2     1
Name: count, dtype: int64
1 [0]


...

predicted_class
4    38
7     6
8     3
3     2
Name: count, dtype: int64
4 [1]


100%|██████████| 16/16 [02:24<00:00,  9.01s/it]



Fold 5 — Accuracy: 0.62, F1 Score: 0.60
              precision    recall  f1-score   support

           0       0.74      0.82      0.78        28
           1       1.00      0.02      0.04        45
           2       0.63      0.50      0.56        34
           3       0.73      0.79      0.76        82
           4       0.54      0.63      0.58        65
           5       0.00      0.00      0.00         6
           6       0.09      0.20      0.12        10
           7       0.62      0.75      0.68        60
           8       0.55      0.62      0.58        56
           9       0.90      0.54      0.68        48
          10       0.94      1.00      0.97        32

    accuracy                           0.62       466
   macro avg       0.61      0.53      0.52       466
weighted avg       0.69      0.62      0.60       466



100%|██████████| 16/16 [02:05<00:00,  7.84s/it]

predicted_class
4     33
7     18
10     2
8      1
Name: count, dtype: int64

Average Accuracy (only folds using 2023 test data): 0.64 (0.04)
Average F1 Score: 0.64 (0.04)


In [92]:
df23_all_preds['class'] = df23_all_preds['predicted_class'].astype(int)
dfallsemisuperivised = pd.concat([gdf18[['class', 'cluster']+wavelength23_cols], gdf23.loc[gdf23['change']==0][['class', 'cluster']+wavelength23_cols], df23_all_preds[['class', 'cluster']+wavelength23_cols]], ignore_index=True)

In [93]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import pandas as pd
import numpy as np

# Combine for consistent group splitting
X_all = dfallsemisuperivised[wavelength23_cols]
y_all = dfallsemisuperivised['class'].astype(int)
groups_all = dfallsemisuperivised['cluster']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)

# Metrics
f1_scores = []
accuracy_scores = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = dfallsemisuperivised.iloc[train_idx]
    test_data = dfallsemisuperivised.iloc[test_idx]

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy: {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")


100%|██████████| 16/16 [02:30<00:00,  9.38s/it]



Fold 1 — Accuracy: 0.64, F1 Score: 0.64
              precision    recall  f1-score   support

           0       0.82      0.58      0.68        24
           1       0.14      1.00      0.25         2
           2       0.37      0.60      0.46        48
           3       0.57      0.74      0.64       148
           4       0.63      0.73      0.68       147
           5       0.79      0.43      0.56        72
           6       0.83      0.69      0.75       103
           7       0.80      0.72      0.76        85
           8       0.66      0.45      0.54       185
           9       0.66      0.81      0.72        26
          10       0.70      1.00      0.82        16

    accuracy                           0.64       856
   macro avg       0.63      0.71      0.62       856
weighted avg       0.67      0.64      0.64       856



100%|██████████| 16/16 [02:33<00:00,  9.61s/it]



Fold 2 — Accuracy: 0.69, F1 Score: 0.71
              precision    recall  f1-score   support

           0       0.82      0.64      0.72        22
           1       0.00      0.00      0.00         0
           2       0.91      0.70      0.79       198
           3       0.41      0.59      0.48        90
           4       0.74      0.69      0.71       107
           5       0.70      0.80      0.75        93
           6       0.69      0.73      0.71        45
           7       0.92      0.81      0.86        27
           8       0.55      0.53      0.54        97
           9       0.00      0.00      0.00         0
          10       1.00      0.94      0.97        48

    accuracy                           0.69       727
   macro avg       0.61      0.58      0.59       727
weighted avg       0.74      0.69      0.71       727



100%|██████████| 16/16 [02:37<00:00,  9.87s/it]



Fold 3 — Accuracy: 0.65, F1 Score: 0.66
              precision    recall  f1-score   support

           0       0.84      0.81      0.83        47
           1       0.25      0.05      0.09        38
           2       0.40      0.66      0.49        32
           3       0.88      0.60      0.71       201
           4       0.64      0.84      0.72       130
           5       0.50      0.61      0.55        28
           6       0.39      0.43      0.41        28
           7       0.72      0.69      0.70        48
           8       0.33      0.39      0.36        18
           9       0.00      0.00      0.00         0
          10       0.98      1.00      0.99        41

    accuracy                           0.65       611
   macro avg       0.54      0.55      0.53       611
weighted avg       0.70      0.65      0.66       611



100%|██████████| 16/16 [02:37<00:00,  9.87s/it]



Fold 4 — Accuracy: 0.72, F1 Score: 0.73
              precision    recall  f1-score   support

           0       0.92      0.69      0.79        16
           1       0.00      0.00      0.00         0
           2       0.91      0.66      0.77       148
           3       0.75      0.81      0.78       128
           4       0.78      0.82      0.79       114
           5       0.06      0.25      0.09         4
           6       0.62      0.56      0.59        52
           7       0.60      0.86      0.71        28
           8       0.44      0.68      0.53        41
           9       0.83      0.36      0.50        42
          10       0.96      1.00      0.98        24

    accuracy                           0.72       597
   macro avg       0.62      0.61      0.59       597
weighted avg       0.77      0.72      0.73       597



100%|██████████| 16/16 [02:40<00:00, 10.05s/it]


Fold 5 — Accuracy: 0.65, F1 Score: 0.63
              precision    recall  f1-score   support

           0       0.77      0.82      0.79        28
           1       1.00      0.02      0.04        45
           2       0.63      0.50      0.56        34
           3       0.71      0.79      0.75        82
           4       0.66      0.74      0.70        98
           5       0.00      0.00      0.00         6
           6       0.09      0.20      0.12        10
           7       0.69      0.79      0.74        78
           8       0.54      0.61      0.57        57
           9       0.89      0.52      0.66        48
          10       0.94      1.00      0.97        34

    accuracy                           0.65       520
   macro avg       0.63      0.55      0.54       520
weighted avg       0.71      0.65      0.63       520


Average Accuracy: 0.67 (0.03)
Average F1 Score: 0.67 (0.04)


In [14]:
import joblib 

# Train model on all data for deployment
combined = pd.concat([gdf18, gdf23.loc[gdf23['change']==0]], ignore_index=True)
combinednorm = l2_normalize_dataframe(combined, wavelength23_cols)
X = combinednorm[wavelength23_cols]
y = combinednorm['class'].astype(int)

# Classifier
# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)
clf.fit(X, y)

# Get pseudo labels for 2023 change points
df23changesubset = gdf23[(gdf23['change'] == 1)].copy()
X = df23changesubset[wavelength23_cols]
predicted_labels = clf.predict(X)
df23changesubset['predicted_class'] = predicted_labels

df23changesubset['class'] = df23changesubset['predicted_class'].astype(int)
dfallsemisuperivised = pd.concat([gdf18[['class', 'cluster']+wavelength23_cols], gdf23.loc[gdf23['change']==0][['class', 'cluster']+wavelength23_cols], df23changesubset[['class', 'cluster']+wavelength23_cols]], ignore_index=True)
X = dfallsemisuperivised[wavelength23_cols]
y = dfallsemisuperivised['class'].astype(int)
sslbase_clf = TabPFNClassifier()

# Initialize a classifier
sslclf = ManyClassClassifier(
    estimator= sslbase_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)
sslclf.fit(X, y)
# -------------------------------------------------
# 2. Export the trained model to a file
# -------------------------------------------------

model_path = r"C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E51_2018_TABPFN.pkl"   # choose any path you like
joblib.dump(sslclf, model_path)

print(f"Model saved to {model_path}")

100%|██████████| 16/16 [02:56<00:00, 11.04s/it]

Model saved to C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E51_2018_TABPFN.pkl


### Experiment 5.2: SSL+Imad filtering

In [15]:
# with spatial CV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import pandas as pd
import numpy as np

# Add source labels
gdf18['source'] = '2018'
gdf23['source'] = '2023'

# Combine for consistent group splitting
combined = pd.concat([gdf18, gdf23.loc[gdf23['imad_change']==1]], ignore_index=True)
X_all = combined[wavelength23_cols]
y_all = combined['class'].astype(int)
groups_all = combined['cluster']

# Track source to separate later
sources = combined['source']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Metrics
f1_scores = []
accuracy_scores = []

# Collect all 2023 change subset predictions
df23_all_preds = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = combined.iloc[train_idx]
    test_data = combined.iloc[test_idx]

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)
    print(i , groups_all.iloc[test_idx].unique())

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

    # Get pseudo labels for 2023 change points
    df23changesubset = gdf23[(gdf23['imad_change'] == 0) & (gdf23['cluster']==groups_all.iloc[test_idx].unique()[0])].copy()
    X = df23changesubset[wavelength23_cols]
    predicted_labels = clf.predict(X)
    df23changesubset['predicted_class'] = predicted_labels
    df23_all_preds.append(df23changesubset)
    print(df23changesubset['predicted_class'].value_counts())

df23_all_preds = pd.concat(df23_all_preds, ignore_index=False)

# Summary
print(f"\nAverage Accuracy (only folds using 2023 test data): {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")


0 [3]

Fold 1 — Accuracy: 0.43, F1 Score: 0.41
              precision    recall  f1-score   support

           0       0.40      0.15      0.22        13
           1       0.00      0.00      0.00         0
           2       0.24      0.46      0.31        39
           3       0.41      0.74      0.53       125
           4       0.44      0.51      0.47        88
           5       0.85      0.23      0.37        47
           6       0.67      0.30      0.42        73
           7       0.68      0.42      0.52        50
           8       0.54      0.16      0.25       124
           9       0.47      0.55      0.51        31
          10       0.36      1.00      0.53         8

    accuracy                           0.43       598
   macro avg       0.46      0.41      0.37       598
weighted avg       0.52      0.43      0.41       598

predicted_class
4     80
8     47
3     45
7     22
6     19
0     11
10    10
2      9
1      6
9      5
5      4
Name: count, dtype: int64

In [11]:
df23_all_preds['class'] = df23_all_preds['predicted_class'].astype(int)
dfallsemisuperivised = pd.concat([gdf18[['class', 'cluster']+wavelength23_cols], gdf23.loc[gdf23['imad_change']==1][['class', 'cluster']+wavelength23_cols], df23_all_preds[['class', 'cluster']+wavelength23_cols]], ignore_index=True)

In [12]:
# Combine for consistent group splitting
X_all = dfallsemisuperivised[wavelength23_cols]
y_all = dfallsemisuperivised['class'].astype(int)
groups_all = dfallsemisuperivised['cluster']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Metrics
f1_scores = []
accuracy_scores = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = dfallsemisuperivised.iloc[train_idx]
    test_data = dfallsemisuperivised.iloc[test_idx]

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy: {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")


Fold 1 — Accuracy: 0.61, F1 Score: 0.60
              precision    recall  f1-score   support

           0       0.87      0.54      0.67        24
           1       0.28      0.83      0.42         6
           2       0.33      0.56      0.42        48
           3       0.53      0.79      0.63       170
           4       0.68      0.76      0.72       168
           5       0.88      0.27      0.42        51
           6       0.75      0.50      0.60        92
           7       0.78      0.64      0.70        72
           8       0.71      0.40      0.51       171
           9       0.59      0.64      0.61        36
          10       0.55      0.94      0.69        18

    accuracy                           0.61       856
   macro avg       0.63      0.63      0.58       856
weighted avg       0.66      0.61      0.60       856


Fold 2 — Accuracy: 0.67, F1 Score: 0.68
              precision    recall  f1-score   support

           0       1.00      0.50      0.67       

c:\ProgramData\Anaconda3\envs\erthy\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\ProgramData\Anaconda3\envs\erthy\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\ProgramData\Anaconda3\envs\erthy\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Fold 3 — Accuracy: 0.63, F1 Score: 0.65
              precision    recall  f1-score   support

           0       0.97      0.85      0.91        41
           1       0.88      0.27      0.41        26
           2       0.34      0.36      0.35        28
           3       0.74      0.60      0.66       197
           4       0.65      0.76      0.70       140
           5       0.82      0.38      0.51        24
           6       0.57      0.52      0.54        31
           7       0.71      0.63      0.67        43
           8       0.32      0.42      0.36        36
           9       0.08      0.67      0.14         6
          10       0.97      0.97      0.97        39

    accuracy                           0.63       611
   macro avg       0.64      0.58      0.57       611
weighted avg       0.70      0.63      0.65       611


Fold 4 — Accuracy: 0.69, F1 Score: 0.67
              precision    recall  f1-score   support

           0       0.91      0.71      0.80       

In [16]:
import joblib 

# Train model on all data for deployment
combined = pd.concat([gdf18, gdf23.loc[gdf23['imad_change']==1]], ignore_index=True)
X = combined[wavelength23_cols]
y = combined['class'].astype(int)

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)
clf.fit(X, y)

# Get pseudo labels for 2023 change points
df23changesubset = gdf23[gdf23['imad_change']==0].copy()
X = df23changesubset[wavelength23_cols]
predicted_labels = clf.predict(X)
df23changesubset['predicted_class'] = predicted_labels

df23changesubset['class'] = df23changesubset['predicted_class'].astype(int)
dfallsemisuperivised = pd.concat([gdf18[['class', 'cluster']+wavelength23_cols], gdf23.loc[gdf23['imad_change']==1][['class', 'cluster']+wavelength23_cols], df23changesubset[['class', 'cluster']+wavelength23_cols]], ignore_index=True)
X = dfallsemisuperivised[wavelength23_cols]
y = dfallsemisuperivised['class'].astype(int)
sslclf = RandomForestClassifier(random_state=42, n_jobs=-1)
sslclf.fit(X, y)
# -------------------------------------------------
# 2. Export the trained model to a file
# -------------------------------------------------

model_path = r"C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E52_2018_RF.pkl"   # choose any path you like
joblib.dump(sslclf, model_path)

print(f"Model saved to {model_path}")

Model saved to C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E52_2018_RF.pkl


TABPFN

In [21]:
# with spatial CV

# Add source labels
gdf18['source'] = '2018'
gdf23['source'] = '2023'

# Combine for consistent group splitting
combined = pd.concat([gdf18, gdf23.loc[gdf23['imad_change']==1]], ignore_index=True)
X_all = combined[wavelength23_cols]
y_all = combined['class'].astype(int)
groups_all = combined['cluster']

# Track source to separate later
sources = combined['source']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)

# Metrics
f1_scores = []
accuracy_scores = []

# Collect all 2023 change subset predictions
df23_all_preds = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = combined.iloc[train_idx]
    test_data = combined.iloc[test_idx]

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)
    print(i , groups_all.iloc[test_idx].unique())

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

    # Get pseudo labels for 2023 change points
    df23changesubset = gdf23[(gdf23['imad_change'] == 0) & (gdf23['cluster']==groups_all.iloc[test_idx].unique()[0])].copy()
    X = df23changesubset[wavelength23_cols]
    predicted_labels = clf.predict(X)
    df23changesubset['predicted_class'] = predicted_labels
    df23_all_preds.append(df23changesubset)
    print(df23changesubset['predicted_class'].value_counts())

df23_all_preds = pd.concat(df23_all_preds, ignore_index=False)

# Summary
print(f"\nAverage Accuracy (only folds using 2023 test data): {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")


0 [3]


100%|██████████| 16/16 [01:29<00:00,  5.62s/it]



Fold 1 — Accuracy: 0.61, F1 Score: 0.61
              precision    recall  f1-score   support

           0       0.88      0.54      0.67        13
           1       0.00      0.00      0.00         0
           2       0.43      0.72      0.54        39
           3       0.57      0.71      0.63       125
           4       0.52      0.61      0.57        88
           5       0.76      0.40      0.53        47
           6       0.80      0.70      0.74        73
           7       0.83      0.58      0.68        50
           8       0.64      0.45      0.53       124
           9       0.66      0.74      0.70        31
          10       0.57      1.00      0.73         8

    accuracy                           0.61       598
   macro avg       0.61      0.59      0.57       598
weighted avg       0.64      0.61      0.61       598



100%|██████████| 16/16 [01:20<00:00,  5.01s/it]


predicted_class
4     76
8     51
7     32
3     31
6     26
2     10
10    10
0      7
5      7
1      5
9      3
Name: count, dtype: int64
1 [0]


...

predicted_class
4     35
8     27
3     21
6     16
10     8
2      7
7      7
0      5
5      4
1      1
Name: count, dtype: int64
4 [1]


100%|██████████| 16/16 [01:36<00:00,  6.06s/it]



Fold 5 — Accuracy: 0.62, F1 Score: 0.61
              precision    recall  f1-score   support

           0       0.74      0.88      0.80        16
           1       0.60      0.12      0.20        25
           2       0.60      0.40      0.48        30
           3       0.73      0.80      0.76        66
           4       0.46      0.65      0.54        57
           5       0.00      0.00      0.00         4
           6       0.11      0.25      0.15         8
           7       0.57      0.81      0.67        52
           8       0.65      0.63      0.64        38
           9       0.93      0.42      0.58        67
          10       1.00      1.00      1.00        27

    accuracy                           0.62       390
   macro avg       0.58      0.54      0.53       390
weighted avg       0.68      0.62      0.61       390



100%|██████████| 16/16 [01:28<00:00,  5.52s/it]

predicted_class
8     35
4     25
7     22
3     16
0     10
10     9
6      4
5      3
2      3
9      2
1      1
Name: count, dtype: int64

Average Accuracy (only folds using 2023 test data): 0.63 (0.04)
Average F1 Score: 0.64 (0.03)


In [22]:
df23_all_preds['class'] = df23_all_preds['predicted_class'].astype(int)
dfallsemisuperivised = pd.concat([gdf18[['class', 'cluster']+wavelength23_cols], gdf23.loc[gdf23['imad_change']==1][['class', 'cluster']+wavelength23_cols], df23_all_preds[['class', 'cluster']+wavelength23_cols]], ignore_index=True)

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import pandas as pd
import numpy as np

# Combine for consistent group splitting
X_all = dfallsemisuperivised[wavelength23_cols]
y_all = dfallsemisuperivised['class'].astype(int)
groups_all = dfallsemisuperivised['cluster']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)

# Metrics
f1_scores = []
accuracy_scores = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = dfallsemisuperivised.iloc[train_idx]
    test_data = dfallsemisuperivised.iloc[test_idx]

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy: {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")


100%|██████████| 16/16 [02:23<00:00,  8.96s/it]



Fold 1 — Accuracy: 0.73, F1 Score: 0.73
              precision    recall  f1-score   support

           0       0.82      0.70      0.76        20
           1       0.43      0.60      0.50         5
           2       0.50      0.73      0.60        49
           3       0.63      0.78      0.70       156
           4       0.75      0.79      0.77       164
           5       0.88      0.54      0.67        54
           6       0.84      0.82      0.83        99
           7       0.87      0.79      0.83        82
           8       0.79      0.59      0.67       175
           9       0.66      0.68      0.67        34
          10       0.75      1.00      0.86        18

    accuracy                           0.73       856
   macro avg       0.72      0.73      0.71       856
weighted avg       0.75      0.73      0.73       856



100%|██████████| 16/16 [02:28<00:00,  9.26s/it]



Fold 2 — Accuracy: 0.77, F1 Score: 0.78
              precision    recall  f1-score   support

           0       1.00      0.68      0.81        22
           1       0.38      1.00      0.55         3
           2       0.93      0.79      0.85       174
           3       0.63      0.69      0.66       116
           4       0.83      0.77      0.80       113
           5       0.72      0.92      0.81        77
           6       0.75      0.80      0.78        50
           7       0.88      0.75      0.81        28
           8       0.64      0.66      0.65        99
           9       0.25      1.00      0.40         1
          10       0.95      0.95      0.95        44

    accuracy                           0.77       727
   macro avg       0.72      0.82      0.73       727
weighted avg       0.79      0.77      0.78       727



100%|██████████| 16/16 [02:32<00:00,  9.56s/it]



Fold 3 — Accuracy: 0.70, F1 Score: 0.71
              precision    recall  f1-score   support

           0       0.95      0.86      0.90        42
           1       0.75      0.25      0.38        24
           2       0.44      0.53      0.48        40
           3       0.82      0.65      0.72       191
           4       0.68      0.83      0.75       141
           5       0.78      0.48      0.60        29
           6       0.67      0.57      0.62        35
           7       0.71      0.71      0.71        45
           8       0.42      0.59      0.49        22
           9       0.00      0.00      0.00         0
          10       1.00      1.00      1.00        42

    accuracy                           0.70       611
   macro avg       0.66      0.59      0.60       611
weighted avg       0.75      0.70      0.71       611



100%|██████████| 16/16 [02:33<00:00,  9.59s/it]



Fold 4 — Accuracy: 0.73, F1 Score: 0.73
              precision    recall  f1-score   support

           0       1.00      0.93      0.96        14
           1       0.00      0.00      0.00         1
           2       0.91      0.74      0.82       129
           3       0.75      0.85      0.80       128
           4       0.66      0.78      0.71        98
           5       0.33      0.50      0.40         6
           6       0.64      0.74      0.69        47
           7       0.51      0.77      0.62        26
           8       0.68      0.78      0.73        58
           9       0.82      0.27      0.41        66
          10       0.96      0.92      0.94        24

    accuracy                           0.73       597
   macro avg       0.66      0.66      0.64       597
weighted avg       0.76      0.73      0.73       597



100%|██████████| 16/16 [02:36<00:00,  9.79s/it]


Fold 5 — Accuracy: 0.71, F1 Score: 0.70
              precision    recall  f1-score   support

           0       0.80      0.92      0.86        26
           1       0.82      0.35      0.49        26
           2       0.54      0.58      0.56        33
           3       0.73      0.80      0.77        82
           4       0.56      0.73      0.63        82
           5       0.38      0.43      0.40         7
           6       0.31      0.42      0.36        12
           7       0.70      0.85      0.77        74
           8       0.81      0.70      0.75        73
           9       0.94      0.45      0.61        69
          10       1.00      1.00      1.00        36

    accuracy                           0.71       520
   macro avg       0.69      0.66      0.65       520
weighted avg       0.74      0.71      0.70       520


Average Accuracy: 0.73 (0.03)
Average F1 Score: 0.73 (0.03)


In [17]:
import joblib 

# Train model on all data for deployment
combined = pd.concat([gdf18, gdf23.loc[gdf23['imad_change']==1]], ignore_index=True)
X = combined[wavelength23_cols]
y = combined['class'].astype(int)

# Create a base TabPFN classifier
base_clf = TabPFNClassifier()
# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)
clf.fit(X, y)

# Get pseudo labels for 2023 change points
df23changesubset = gdf23[gdf23['imad_change']==0].copy()
X = df23changesubset[wavelength23_cols]
predicted_labels = clf.predict(X)
df23changesubset['predicted_class'] = predicted_labels

df23changesubset['class'] = df23changesubset['predicted_class'].astype(int)
dfallsemisuperivised = pd.concat([gdf18[['class', 'cluster']+wavelength23_cols], gdf23.loc[gdf23['imad_change']==1][['class', 'cluster']+wavelength23_cols], df23changesubset[['class', 'cluster']+wavelength23_cols]], ignore_index=True)
X = dfallsemisuperivised[wavelength23_cols]
y = dfallsemisuperivised['class'].astype(int)
sslbase_clf = TabPFNClassifier()
# Initialize a classifier
sslclf = ManyClassClassifier(
    estimator= sslbase_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)
sslclf.fit(X, y)
# -------------------------------------------------
# 2. Export the trained model to a file
# -------------------------------------------------

model_path = r"C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E52_2018_TABPFN.pkl"   # choose any path you like
joblib.dump(sslclf, model_path)

print(f"Model saved to {model_path}")

100%|██████████| 16/16 [02:20<00:00,  8.81s/it]

Model saved to C:\Users\coach\myfiles\postdoc\Invasives\models\CS1_S2_E52_2018_TABPFN.pkl


### Experiment 1.5: train on 2018 (no change), predict on 2018 (no change)

In [14]:
df18subset = gdf18[gdf18['change'] == 0].copy()

In [15]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import numpy as np

# Features and target
X = df18subset[wavelength23_cols]
y = df18subset['class'].astype(int)
groups = df18subset['cluster']  # replace with the appropriate grouping column

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)

# GroupKFold setup
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Store metrics
f1_scores = []
accuracy_scores = []
reports = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X, y, groups)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # Accuracy
    acc = accuracy_score(y_test, y_pred)
    accuracy_scores.append(acc)

    # F1
    f1 = f1_score(y_test, y_pred, average='weighted')
    f1_scores.append(f1)
    
    # Report
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)
    
    print(f"\nFold {i + 1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy over {n_splits} folds: {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score over {n_splits} folds: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")



Fold 1 — Accuracy: 0.41, F1 Score: 0.41
              precision    recall  f1-score   support

           0       0.67      0.33      0.44        12
           1       0.00      0.00      0.00         0
           2       0.09      0.27      0.14        15
           3       0.32      0.69      0.44        64
           4       0.49      0.53      0.51        58
           5       0.80      0.35      0.49        34
           6       0.76      0.25      0.38        51
           7       0.71      0.41      0.52        37
           8       0.59      0.21      0.31        90
          10       0.35      1.00      0.52         8

    accuracy                           0.41       369
   macro avg       0.48      0.40      0.37       369
weighted avg       0.56      0.41      0.41       369


Fold 2 — Accuracy: 0.55, F1 Score: 0.54
              precision    recall  f1-score   support

           0       0.80      0.87      0.83        23
           1       0.67      0.11      0.18       

In [37]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

# Load data
X, y = df18subset[wavelength23_cols], df18subset['class'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

clf.fit(X_train, y_train)

RandomForestClassifier()

In [38]:
prediction_probabilities = clf.predict_proba(X_test)
prediction_probabilities.shape

# Get the predicted class labels
predicted_labels = clf.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

# from .65

Accuracy: 0.63


In [17]:
from tabpfn import TabPFNClassifier
from tabpfn_extensions.many_class import ManyClassClassifier 
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import numpy as np

# Features and target
X = gdf18[wavelength23_cols]
y = gdf18['class'].astype(int)
groups = gdf18['cluster']  # replace with the appropriate grouping column

# Classifier
# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)

# GroupKFold setup
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Store metrics
f1_scores = []
accuracy_scores = []
reports = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X, y, groups)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # Accuracy
    acc = accuracy_score(y_test, y_pred)
    accuracy_scores.append(acc)

    # F1
    f1 = f1_score(y_test, y_pred, average='weighted')
    f1_scores.append(f1)
    
    # Report
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)
    
    print(f"\nFold {i + 1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy over {n_splits} folds: {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score over {n_splits} folds: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")


  0%|          | 0/16 [00:00<?, ?it/s]

(…)fn-v2-classifier-finetuned-zk73skhh.ckpt:   0%|          | 0.00/29.0M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/37.0 [00:00<?, ?B/s]

100%|██████████| 16/16 [01:18<00:00,  4.89s/it]



Fold 1 — Accuracy: 0.62, F1 Score: 0.63
              precision    recall  f1-score   support

           0       0.90      0.75      0.82        12
           1       0.00      0.00      0.00         0
           2       0.40      0.56      0.47        32
           3       0.53      0.71      0.61        78
           4       0.65      0.71      0.68        65
           5       0.76      0.42      0.54        38
           6       0.83      0.73      0.78        52
           7       0.91      0.54      0.68        37
           8       0.66      0.44      0.53        91
           9       0.64      0.92      0.75        25
          10       0.53      1.00      0.70         8

    accuracy                           0.62       438
   macro avg       0.62      0.62      0.59       438
weighted avg       0.67      0.62      0.63       438



...


Fold 3 — Accuracy: 0.66, F1 Score: 0.65
              precision    recall  f1-score   support

           0       0.69      0.87      0.77        23
           1       0.50      0.05      0.10        19
           2       0.60      0.75      0.67        16
           3       0.84      0.73      0.78       102
           4       0.63      0.66      0.65        65
           5       0.42      0.36      0.38        14
           6       0.21      0.29      0.24        14
           7       0.53      0.79      0.63        24
           8       0.42      0.56      0.48         9
           9       0.00      0.00      0.00         0
          10       1.00      0.95      0.98        21

    accuracy                           0.66       307
   macro avg       0.53      0.55      0.52       307
weighted avg       0.68      0.66      0.65       307



100%|██████████| 16/16 [01:19<00:00,  4.95s/it]



Fold 4 — Accuracy: 0.77, F1 Score: 0.78
              precision    recall  f1-score   support

           0       1.00      0.78      0.88         9
           1       0.00      0.00      0.00         0
           2       0.88      0.80      0.83        79
           3       0.81      0.74      0.77        65
           4       0.71      0.74      0.72        39
           5       0.00      0.00      0.00         2
           6       0.70      0.73      0.72        26
           7       0.46      0.55      0.50        11
           8       0.52      0.74      0.61        19
           9       0.86      0.86      0.86        42
          10       0.92      1.00      0.96        12

    accuracy                           0.77       304
   macro avg       0.62      0.63      0.62       304
weighted avg       0.79      0.77      0.78       304



100%|██████████| 16/16 [01:23<00:00,  5.19s/it]


Fold 5 — Accuracy: 0.68, F1 Score: 0.69
              precision    recall  f1-score   support

           0       0.82      0.93      0.88        15
           1       0.80      0.17      0.28        24
           2       0.75      0.53      0.62        17
           3       0.81      0.91      0.86        43
           4       0.54      0.74      0.62        34
           5       0.00      0.00      0.00         3
           6       0.12      0.40      0.18         5
           7       0.62      0.70      0.66        30
           8       0.71      0.61      0.65        28
           9       0.91      0.67      0.77        48
          10       1.00      1.00      1.00        16

    accuracy                           0.68       263
   macro avg       0.64      0.60      0.59       263
weighted avg       0.75      0.68      0.69       263


Average Accuracy over 5 folds: 0.68 (0.05)
Average F1 Score over 5 folds: 0.68 (0.05)


In [39]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from tabpfn import TabPFNClassifier
from tabpfn_extensions.many_class import ManyClassClassifier

# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Load data
X, y = df18subset[wavelength23_cols], df18subset['class'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)
clf.fit(X_train, y_train)

ManyClassClassifier(alphabet_size=10, estimator=TabPFNClassifier())

In [40]:
# Predict probabilities
# prediction_probabilities = clf.predict_proba(X_test)
# print("ROC AUC:", roc_auc_score(y_test, prediction_probabilities[:, 1]))

# Predict labels
predictions = clf.predict(X_test)
print("Accuracy", accuracy_score(y_test, predictions))

# from .785

Accuracy 0.7417503586800573


### Experiment 1.6: train on 2023 (all), predict on 2023 (all)

In [19]:
# use 2018 class for 2023 whereever there is no change
gdf23.loc[gdf23['change'] == 0, '2023_class'] = gdf23['class']
gdf23.dropna(subset=['2023_class'], inplace=True)

In [21]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import numpy as np

# Features and target
X, y = gdf23[wavelength23_cols], gdf23['2023_class'].astype(int)
groups = gdf23['cluster']  # replace with the appropriate grouping column

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)

# GroupKFold setup
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Store metrics
f1_scores = []
accuracy_scores = []
reports = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X, y, groups)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # Accuracy
    acc = accuracy_score(y_test, y_pred)
    accuracy_scores.append(acc)

    # F1
    f1 = f1_score(y_test, y_pred, average='weighted')
    f1_scores.append(f1)
    
    # Report
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)
    
    print(f"\nFold {i + 1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy over {n_splits} folds: {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score over {n_splits} folds: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")



Fold 1 — Accuracy: 0.50, F1 Score: 0.50
              precision    recall  f1-score   support

           0       0.42      0.64      0.51        25
           1       0.00      0.00      0.00         0
           2       0.10      0.33      0.15        15
           3       0.48      0.61      0.53        64
           4       0.59      0.64      0.61        84
           5       0.76      0.38      0.51        34
           6       0.78      0.35      0.49        51
           7       0.62      0.74      0.67        46
           8       0.57      0.33      0.42        91
          10       0.36      1.00      0.53         8
          12       0.00      0.00      0.00        18

    accuracy                           0.50       436
   macro avg       0.43      0.46      0.40       436
weighted avg       0.55      0.50      0.50       436


Fold 2 — Accuracy: 0.44, F1 Score: 0.44
              precision    recall  f1-score   support

           0       0.50      0.25      0.33       

In [42]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

# Load data
X, y = gdf23[wavelength23_cols], gdf23['2023_class'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

clf.fit(X_train, y_train)

RandomForestClassifier()

In [44]:
prediction_probabilities = clf.predict_proba(X_test)
prediction_probabilities.shape

# Get the predicted class labels
predicted_labels = clf.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

# from .54

Accuracy: 0.59


In [22]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import numpy as np

# Features and target
X, y = gdf23[wavelength23_cols], gdf23['2023_class'].astype(int)
groups = gdf23['cluster']  # replace with the appropriate grouping column

# Classifier
# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)

# GroupKFold setup
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Store metrics
f1_scores = []
accuracy_scores = []
reports = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X, y, groups)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # Accuracy
    acc = accuracy_score(y_test, y_pred)
    accuracy_scores.append(acc)

    # F1
    f1 = f1_score(y_test, y_pred, average='weighted')
    f1_scores.append(f1)
    
    # Report
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)
    
    print(f"\nFold {i + 1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy over {n_splits} folds: {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score over {n_splits} folds: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")


100%|██████████| 16/16 [01:12<00:00,  4.53s/it]



Fold 1 — Accuracy: 0.58, F1 Score: 0.58
              precision    recall  f1-score   support

           0       0.38      0.44      0.41        25
           1       0.00      0.00      0.00         0
           2       0.24      0.53      0.33        15
           3       0.54      0.69      0.60        64
           4       0.60      0.65      0.62        84
           5       0.90      0.56      0.69        34
           6       0.88      0.69      0.77        51
           7       0.64      0.78      0.71        46
           8       0.64      0.41      0.50        91
          10       0.40      1.00      0.57         8
          12       0.00      0.00      0.00        18

    accuracy                           0.58       436
   macro avg       0.47      0.52      0.47       436
weighted avg       0.61      0.58      0.58       436



100%|██████████| 16/16 [01:17<00:00,  4.84s/it]



Fold 2 — Accuracy: 0.53, F1 Score: 0.52
              precision    recall  f1-score   support

           0       0.48      0.19      0.27        52
           1       0.00      0.00      0.00         0
           2       0.66      0.57      0.61        68
           3       0.26      0.52      0.34        31
           4       0.30      0.66      0.41        32
           5       0.65      0.67      0.66        45
           6       0.77      0.91      0.83        22
           7       0.79      0.41      0.54        27
           8       0.64      0.57      0.61        47
          10       0.96      0.89      0.92        27
          12       0.00      0.00      0.00        25

    accuracy                           0.53       376
   macro avg       0.50      0.49      0.47       376
weighted avg       0.56      0.53      0.52       376



100%|██████████| 16/16 [01:21<00:00,  5.07s/it]



Fold 3 — Accuracy: 0.59, F1 Score: 0.59
              precision    recall  f1-score   support

           0       0.33      0.80      0.47        25
           1       0.00      0.00      0.00        19
           2       0.43      0.75      0.55        16
           3       0.89      0.52      0.66       103
           4       0.66      0.55      0.60        60
           5       0.41      0.64      0.50        14
           6       0.60      0.64      0.62        14
           7       0.71      0.83      0.77        24
           8       0.50      0.56      0.53         9
          10       0.91      1.00      0.95        20
          12       0.00      0.00      0.00         3

    accuracy                           0.59       307
   macro avg       0.49      0.57      0.51       307
weighted avg       0.65      0.59      0.59       307



100%|██████████| 16/16 [01:22<00:00,  5.16s/it]



Fold 4 — Accuracy: 0.66, F1 Score: 0.67
              precision    recall  f1-score   support

           0       0.67      0.60      0.63        10
           1       0.00      0.00      0.00         0
           2       0.95      0.55      0.70        69
           3       0.74      0.77      0.76        66
           4       0.64      0.77      0.70        57
           5       0.11      0.50      0.18         2
           6       0.62      0.69      0.65        26
           7       0.75      0.50      0.60        24
           8       0.43      0.67      0.52        27
          10       0.92      1.00      0.96        12
          12       0.17      0.09      0.12        11

    accuracy                           0.66       304
   macro avg       0.54      0.56      0.53       304
weighted avg       0.71      0.66      0.67       304



100%|██████████| 16/16 [01:26<00:00,  5.43s/it]


Fold 5 — Accuracy: 0.54, F1 Score: 0.55
              precision    recall  f1-score   support

           0       0.64      0.56      0.60        16
           1       0.00      0.00      0.00        21
           2       0.50      0.35      0.41        17
           3       0.71      0.27      0.39        44
           4       0.82      0.68      0.74        59
           5       0.03      0.33      0.05         3
           6       0.05      0.20      0.08         5
           7       0.64      0.83      0.72        42
           8       0.53      0.66      0.58        32
          10       0.90      1.00      0.95        18
          12       0.00      0.00      0.00         6

    accuracy                           0.54       263
   macro avg       0.44      0.44      0.41       263
weighted avg       0.60      0.54      0.55       263


Average Accuracy over 5 folds: 0.58 (0.05)
Average F1 Score over 5 folds: 0.58 (0.05)


In [45]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from tabpfn import TabPFNClassifier
from tabpfn_extensions.many_class import ManyClassClassifier

# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Load data
X, y = gdf23[wavelength23_cols], gdf23['2023_class'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)
clf.fit(X_train, y_train)

ManyClassClassifier(alphabet_size=10, estimator=TabPFNClassifier())

In [46]:
# Predict probabilities
# prediction_probabilities = clf.predict_proba(X_test)
# print("ROC AUC:", roc_auc_score(y_test, prediction_probabilities[:, 1]))

# Predict labels
predictions = clf.predict(X_test)
print("Accuracy", accuracy_score(y_test, predictions))

# from .690

100%|██████████| 16/16 [00:04<00:00,  3.91it/s]

Accuracy 0.6761565836298933


### Experiment 2: train on 2023 (no change), predict on 2023 (no change)

In [ ]:
df23subset = gdf23[gdf23['change'] == 0].copy()
# class and 2023_class are the same for no change points

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import numpy as np

# Features and target
X, y = df23subset[wavelength23_cols], df23subset['class'].astype(int)
groups = df23subset['cluster']  # replace with the appropriate grouping column

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)

# GroupKFold setup
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Store metrics
f1_scores = []
accuracy_scores = []
reports = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X, y, groups)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # Accuracy
    acc = accuracy_score(y_test, y_pred)
    accuracy_scores.append(acc)

    # F1
    f1 = f1_score(y_test, y_pred, average='weighted')
    f1_scores.append(f1)
    
    # Report
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)
    
    print(f"\nFold {i + 1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy over {n_splits} folds: {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score over {n_splits} folds: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")



Fold 1 — Accuracy: 0.56, F1 Score: 0.58
              precision    recall  f1-score   support

           0       0.67      0.67      0.67        12
           1       0.00      0.00      0.00         0
           2       0.10      0.33      0.16        15
           3       0.49      0.56      0.52        64
           4       0.60      0.79      0.68        58
           5       0.83      0.44      0.58        34
           6       0.83      0.39      0.53        51
           7       0.91      0.78      0.84        37
           8       0.66      0.43      0.52        90
          10       0.38      1.00      0.55         8

    accuracy                           0.56       369
   macro avg       0.55      0.54      0.51       369
weighted avg       0.66      0.56      0.58       369


Fold 2 — Accuracy: 0.57, F1 Score: 0.58
              precision    recall  f1-score   support

           0       0.76      0.57      0.65        23
           1       0.11      0.05      0.07       

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

# Load data
X, y = df23subset[wavelength23_cols], df23subset['class'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

clf.fit(X_train, y_train)

RandomForestClassifier()

In [32]:
prediction_probabilities = clf.predict_proba(X_test)
prediction_probabilities.shape

# Get the predicted class labels
predicted_labels = clf.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

# from .60

Accuracy: 0.63


In [ ]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import numpy as np

# Features and target
X, y = df23subset[wavelength23_cols], df23subset['class'].astype(int)
groups = df23subset['cluster']  # replace with the appropriate grouping column

# Classifier
# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)

# GroupKFold setup
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Store metrics
f1_scores = []
accuracy_scores = []
reports = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X, y, groups)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # Accuracy
    acc = accuracy_score(y_test, y_pred)
    accuracy_scores.append(acc)

    # F1
    f1 = f1_score(y_test, y_pred, average='weighted')
    f1_scores.append(f1)
    
    # Report
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)
    
    print(f"\nFold {i + 1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy over {n_splits} folds: {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score over {n_splits} folds: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")



Fold 1 — Accuracy: 0.60, F1 Score: 0.61
              precision    recall  f1-score   support

           0       0.70      0.58      0.64        12
           1       0.00      0.00      0.00         0
           2       0.21      0.60      0.32        15
           3       0.54      0.67      0.60        64
           4       0.60      0.69      0.64        58
           5       0.89      0.47      0.62        34
           6       0.89      0.65      0.75        51
           7       0.72      0.78      0.75        37
           8       0.70      0.41      0.52        90
          10       0.42      1.00      0.59         8

    accuracy                           0.60       369
   macro avg       0.57      0.59      0.54       369
weighted avg       0.68      0.60      0.61       369


Fold 2 — Accuracy: 0.68, F1 Score: 0.68
              precision    recall  f1-score   support

           0       0.78      0.61      0.68        23
           1       0.00      0.00      0.00       

In [51]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from tabpfn import TabPFNClassifier
from tabpfn_extensions.many_class import ManyClassClassifier

# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Load data
X, y = df23subset[wavelength23_cols], df23subset['class'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)
clf.fit(X_train, y_train)

ManyClassClassifier(alphabet_size=10, estimator=TabPFNClassifier())

In [52]:
# Predict probabilities
# prediction_probabilities = clf.predict_proba(X_test)
# print("ROC AUC:", roc_auc_score(y_test, prediction_probabilities[:, 1]))

# Predict labels
predictions = clf.predict(X_test)
print("Accuracy", accuracy_score(y_test, predictions))

# from .760

Accuracy 0.7546628407460545


### Experiment 3: train on 2018 (2018), predict on 2023 (no change)

In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import pandas as pd
import numpy as np

# Add source labels
gdf18['source'] = '2018'
df23subset['source'] = '2023'

# Combine for consistent group splitting
combined = pd.concat([gdf18, df23subset], ignore_index=True)
X_all = combined[wavelength23_cols]
y_all = combined['class'].astype(int)
groups_all = combined['cluster']

# Track source to separate later
sources = combined['source']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Metrics
f1_scores = []
accuracy_scores = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = combined.iloc[train_idx]
    test_data = combined.iloc[test_idx]

    # subset 2018 train data to only include 2018 data
    train_data = train_data.loc[train_data['source'] == '2018']

    # subset 2023 test data to only include 2023 data
    test_data = test_data.loc[test_data['source'] == '2023']
    
    # Ensure all test groups come from df23subset only
    if not all(test_data['source'] == '2023'):
        raise ValueError("Test set contains data from 2018, which is not allowed.")

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy (only folds using 2023 test data): {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")



Fold 1 — Accuracy: 0.50, F1 Score: 0.50
              precision    recall  f1-score   support

           0       0.73      0.67      0.70        12
           1       0.00      0.00      0.00         0
           2       0.43      0.20      0.27        15
           3       0.46      0.70      0.56        64
           4       0.49      0.72      0.59        58
           5       0.78      0.21      0.33        34
           6       0.63      0.24      0.34        51
           7       1.00      0.41      0.58        37
           8       0.55      0.48      0.51        90
          10       0.67      1.00      0.80         8

    accuracy                           0.50       369
   macro avg       0.57      0.46      0.47       369
weighted avg       0.61      0.50      0.50       369


Fold 2 — Accuracy: 0.45, F1 Score: 0.45
              precision    recall  f1-score   support

           0       0.50      0.20      0.29         5
           1       0.00      0.00      0.00       

In [53]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

# Load data
X, y = gdf18[wavelength23_cols], gdf18['class'].astype(int)

clf.fit(X, y)

RandomForestClassifier()

In [54]:
prediction_probabilities = clf.predict_proba(df23subset[wavelength23_cols])
prediction_probabilities.shape

# Get the predicted class labels
predicted_labels = clf.predict(df23subset[wavelength23_cols])
# Calculate accuracy
accuracy = accuracy_score(df23subset['class'], predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

# from .05

Accuracy: 0.54


In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import pandas as pd
import numpy as np

# Add source labels
gdf18['source'] = '2018'
df23subset['source'] = '2023'

# Combine for consistent group splitting
combined = pd.concat([gdf18, df23subset], ignore_index=True)
X_all = combined[wavelength23_cols]
y_all = combined['class'].astype(int)
groups_all = combined['cluster']

# Track source to separate later
sources = combined['source']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)

# Metrics
f1_scores = []
accuracy_scores = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = combined.iloc[train_idx]
    test_data = combined.iloc[test_idx]

    # subset 2018 train data to only include 2018 data
    train_data = train_data.loc[train_data['source'] == '2018']

    # subset 2023 test data to only include 2023 data
    test_data = test_data.loc[test_data['source'] == '2023']
    
    # Ensure all test groups come from df23subset only
    if not all(test_data['source'] == '2023'):
        raise ValueError("Test set contains data from 2018, which is not allowed.")

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy (only folds using 2023 test data): {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")


100%|██████████| 16/16 [01:08<00:00,  4.28s/it]



Fold 1 — Accuracy: 0.50, F1 Score: 0.50
              precision    recall  f1-score   support

           0       0.67      0.67      0.67        12
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00        15
           3       0.46      0.41      0.43        64
           4       0.51      0.52      0.51        58
           5       0.50      0.09      0.15        34
           6       0.70      0.61      0.65        51
           7       0.95      0.57      0.71        37
           8       0.46      0.63      0.54        90
           9       0.00      0.00      0.00         0
          10       0.80      1.00      0.89         8

    accuracy                           0.50       369
   macro avg       0.46      0.41      0.41       369
weighted avg       0.55      0.50      0.50       369



100%|██████████| 16/16 [01:10<00:00,  4.44s/it]



Fold 2 — Accuracy: 0.51, F1 Score: 0.50
              precision    recall  f1-score   support

           0       0.50      0.60      0.55         5
           1       0.00      0.00      0.00         0
           2       1.00      0.16      0.28        68
           3       0.29      0.65      0.40        31
           4       0.50      0.16      0.24        32
           5       0.51      0.58      0.54        45
           6       0.70      0.64      0.67        22
           7       0.82      0.82      0.82        11
           8       0.57      0.85      0.68        47
          10       1.00      0.79      0.88        19

    accuracy                           0.51       280
   macro avg       0.59      0.52      0.51       280
weighted avg       0.67      0.51      0.50       280



100%|██████████| 16/16 [01:18<00:00,  4.92s/it]



Fold 3 — Accuracy: 0.48, F1 Score: 0.50
              precision    recall  f1-score   support

           0       0.27      0.91      0.42        23
           1       0.52      0.79      0.62        19
           2       0.75      0.19      0.30        16
           3       0.82      0.40      0.54        99
           4       0.75      0.25      0.38        60
           5       0.41      0.50      0.45        14
           6       0.20      0.21      0.21        14
           7       0.79      0.62      0.70        24
           8       0.10      0.56      0.18         9
          10       1.00      1.00      1.00        20

    accuracy                           0.48       298
   macro avg       0.56      0.54      0.48       298
weighted avg       0.68      0.48      0.50       298



100%|██████████| 16/16 [01:15<00:00,  4.73s/it]



Fold 4 — Accuracy: 0.54, F1 Score: 0.51
              precision    recall  f1-score   support

           0       0.56      0.71      0.62         7
           1       0.00      0.00      0.00         0
           2       1.00      0.06      0.11        69
           3       0.66      0.85      0.74        61
           4       0.84      0.57      0.68        37
           5       0.00      0.00      0.00         2
           6       0.41      0.46      0.44        26
           7       0.77      0.91      0.83        11
           8       0.32      0.84      0.46        19
          10       1.00      1.00      1.00        12

    accuracy                           0.54       244
   macro avg       0.56      0.54      0.49       244
weighted avg       0.74      0.54      0.51       244



100%|██████████| 16/16 [01:31<00:00,  5.69s/it]


Fold 5 — Accuracy: 0.52, F1 Score: 0.50
              precision    recall  f1-score   support

           0       0.67      0.77      0.71        13
           1       0.00      0.00      0.00        21
           2       1.00      0.06      0.11        17
           3       0.53      0.49      0.51        39
           4       0.71      0.32      0.44        31
           5       0.30      1.00      0.46         3
           6       0.11      0.60      0.18         5
           7       0.83      0.83      0.83        30
           8       0.38      0.68      0.49        28
          10       0.89      1.00      0.94        16

    accuracy                           0.52       203
   macro avg       0.54      0.57      0.47       203
weighted avg       0.59      0.52      0.50       203


Average Accuracy (only folds using 2023 test data): 0.51 (0.02)
Average F1 Score: 0.50 (0.00)


In [55]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from tabpfn import TabPFNClassifier
from tabpfn_extensions.many_class import ManyClassClassifier

# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Load data
X, y = gdf18[wavelength23_cols], gdf18['class'].astype(int)

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)
clf.fit(X, y)

ManyClassClassifier(alphabet_size=10, estimator=TabPFNClassifier())

In [56]:
# Predict probabilities
# prediction_probabilities = clf.predict_proba(X_test)
# print("ROC AUC:", roc_auc_score(y_test, prediction_probabilities[:, 1]))

# Predict labels
predictions = clf.predict(df23subset[wavelength23_cols])
print("Accuracy", accuracy_score(df23subset['class'], predictions))

# from 0.08

100%|██████████| 16/16 [00:06<00:00,  2.52it/s]

Accuracy 0.5767575322812052


### Experiment 4: train on 2018 (all) and 2023 (no change), predict on 2018 and 2023 (no change)

In [42]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import pandas as pd
import numpy as np

# Add source labels
gdf18['source'] = '2018'
df23subset['source'] = '2023'

# Combine for consistent group splitting
combined = pd.concat([gdf18, df23subset], ignore_index=True)
X_all = combined[wavelength23_cols]
y_all = combined['class'].astype(int)
groups_all = combined['cluster']

# Track source to separate later
sources = combined['source']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Metrics
f1_scores = []
accuracy_scores = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = combined.iloc[train_idx]
    test_data = combined.iloc[test_idx]

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy (only folds using 2023 test data): {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")



Fold 1 — Accuracy: 0.50, F1 Score: 0.50
              precision    recall  f1-score   support

           0       0.46      0.46      0.46        24
           1       0.00      0.00      0.00         0
           2       0.23      0.49      0.31        47
           3       0.42      0.68      0.52       142
           4       0.54      0.71      0.61       123
           5       0.77      0.28      0.41        72
           6       0.80      0.39      0.52       103
           7       0.75      0.54      0.63        74
           8       0.64      0.29      0.40       181
           9       0.58      0.76      0.66        25
          10       0.48      1.00      0.65        16

    accuracy                           0.50       807
   macro avg       0.52      0.51      0.47       807
weighted avg       0.59      0.50      0.50       807


Fold 2 — Accuracy: 0.52, F1 Score: 0.54
              precision    recall  f1-score   support

           0       0.73      0.40      0.52       

In [40]:
df1823 = pd.concat([gdf18[['class']+wavelength23_cols], df23subset[['class']+wavelength23_cols]], ignore_index=True)
df1823

n18 = len(gdf18)
n23 = len(df23subset)


In [61]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

# Load data
X, y = df1823[wavelength23_cols], df1823['class'].astype(int)
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(X, y, df1823.index, test_size=0.5, random_state=42)

clf.fit(X_train, y_train)

RandomForestClassifier()

In [62]:
train_from_18 = (idx_train < n18).sum()
train_from_23 = (idx_train >= n18).sum()
test_from_18 = (idx_test < n18).sum()
test_from_23 = (idx_test >= n18).sum()

train_total = len(idx_train)
test_total = len(idx_test)

print("Train proportions:")
print(f"df18: {train_from_18 / train_total:.2f}, df23: {train_from_23 / train_total:.2f}")

print("Test proportions:")
print(f"df18: {test_from_18 / test_total:.2f}, df23: {test_from_23 / test_total:.2f}")


Train proportions:
df18: 0.55, df23: 0.45
Test proportions:
df18: 0.55, df23: 0.45


In [63]:
prediction_probabilities = clf.predict_proba(X_test)
prediction_probabilities.shape

# Get the predicted class labels
predicted_labels = clf.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

# from .62

Accuracy: 0.65


In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import pandas as pd
import numpy as np

# Add source labels
gdf18['source'] = '2018'
df23subset['source'] = '2023'

# Combine for consistent group splitting
combined = pd.concat([gdf18, df23subset], ignore_index=True)
X_all = combined[wavelength23_cols]
y_all = combined['class'].astype(int)
groups_all = combined['cluster']

# Track source to separate later
sources = combined['source']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)

# Metrics
f1_scores = []
accuracy_scores = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = combined.iloc[train_idx]
    test_data = combined.iloc[test_idx]

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy (only folds using 2023 test data): {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")


100%|██████████| 16/16 [02:41<00:00, 10.11s/it]



Fold 1 — Accuracy: 0.64, F1 Score: 0.65
              precision    recall  f1-score   support

           0       0.74      0.58      0.65        24
           1       0.00      0.00      0.00         0
           2       0.34      0.66      0.45        47
           3       0.56      0.70      0.62       142
           4       0.66      0.70      0.68       123
           5       0.80      0.44      0.57        72
           6       0.87      0.70      0.77       103
           7       0.86      0.76      0.81        74
           8       0.73      0.50      0.59       181
           9       0.63      0.88      0.73        25
          10       0.59      1.00      0.74        16

    accuracy                           0.64       807
   macro avg       0.61      0.63      0.60       807
weighted avg       0.70      0.64      0.65       807



100%|██████████| 16/16 [02:52<00:00, 10.79s/it]



Fold 2 — Accuracy: 0.66, F1 Score: 0.68
              precision    recall  f1-score   support

           0       0.93      0.65      0.76        20
           1       0.00      0.00      0.00         0
           2       0.87      0.64      0.74       192
           3       0.34      0.40      0.37        70
           4       0.69      0.62      0.65        81
           5       0.65      0.69      0.67        91
           6       0.64      0.82      0.72        44
           7       0.81      0.95      0.88        22
           8       0.55      0.65      0.60        97
           9       0.00      0.00      0.00         0
          10       1.00      0.95      0.97        39

    accuracy                           0.66       656
   macro avg       0.59      0.58      0.58       656
weighted avg       0.71      0.66      0.68       656



100%|██████████| 16/16 [02:57<00:00, 11.10s/it]



Fold 3 — Accuracy: 0.69, F1 Score: 0.68
              precision    recall  f1-score   support

           0       0.75      0.72      0.73        46
           1       0.20      0.05      0.08        38
           2       0.42      0.78      0.55        32
           3       0.88      0.68      0.77       201
           4       0.68      0.80      0.73       125
           5       0.47      0.64      0.55        28
           6       0.43      0.54      0.48        28
           7       0.65      0.77      0.70        48
           8       0.59      0.56      0.57        18
           9       0.00      0.00      0.00         0
          10       1.00      0.98      0.99        41

    accuracy                           0.69       605
   macro avg       0.55      0.59      0.56       605
weighted avg       0.70      0.69      0.68       605



100%|██████████| 16/16 [03:02<00:00, 11.38s/it]



Fold 4 — Accuracy: 0.74, F1 Score: 0.75
              precision    recall  f1-score   support

           0       0.92      0.69      0.79        16
           1       0.00      0.00      0.00         0
           2       0.88      0.64      0.74       148
           3       0.81      0.77      0.79       126
           4       0.75      0.79      0.77        76
           5       0.09      0.25      0.13         4
           6       0.60      0.62      0.61        52
           7       0.64      0.82      0.72        22
           8       0.48      0.82      0.60        38
           9       0.90      0.88      0.89        42
          10       0.96      1.00      0.98        24

    accuracy                           0.74       548
   macro avg       0.64      0.66      0.64       548
weighted avg       0.78      0.74      0.75       548



100%|██████████| 16/16 [03:05<00:00, 11.57s/it]


Fold 5 — Accuracy: 0.64, F1 Score: 0.64
              precision    recall  f1-score   support

           0       0.82      0.82      0.82        28
           1       0.60      0.07      0.12        45
           2       0.68      0.50      0.58        34
           3       0.80      0.72      0.76        82
           4       0.62      0.69      0.66        65
           5       0.10      0.50      0.17         6
           6       0.15      0.50      0.23        10
           7       0.70      0.75      0.73        60
           8       0.55      0.61      0.58        56
           9       0.87      0.69      0.77        48
          10       0.94      1.00      0.97        32

    accuracy                           0.64       466
   macro avg       0.62      0.62      0.58       466
weighted avg       0.70      0.64      0.64       466


Average Accuracy (only folds using 2023 test data): 0.68 (0.04)
Average F1 Score: 0.68 (0.04)


In [64]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from tabpfn import TabPFNClassifier
from tabpfn_extensions.many_class import ManyClassClassifier

# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Load data
X, y = df1823[wavelength23_cols], df1823['class'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)
clf.fit(X_train, y_train)

ManyClassClassifier(alphabet_size=10, estimator=TabPFNClassifier())

In [65]:
# Predict probabilities
# prediction_probabilities = clf.predict_proba(X_test)
# print("ROC AUC:", roc_auc_score(y_test, prediction_probabilities[:, 1]))

# Predict labels
predictions = clf.predict(X_test)
print("Accuracy", accuracy_score(y_test, predictions))

# from .767

100%|██████████| 16/16 [00:05<00:00,  2.71it/s]

Accuracy 0.7709279688513953


### Experiment 5: train on 2018 (all) and 2023 (no change), predict on 2023 (change). Then train on all 2018 and 2023 and predict on 2023 (all)


In [50]:
# with spatial CV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import pandas as pd
import numpy as np

# Add source labels
gdf18['source'] = '2018'
df23subset['source'] = '2023'

# Combine for consistent group splitting
combined = pd.concat([gdf18, df23subset], ignore_index=True)
X_all = combined[wavelength23_cols]
y_all = combined['class'].astype(int)
groups_all = combined['cluster']

# Track source to separate later
sources = combined['source']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Metrics
f1_scores = []
accuracy_scores = []

# Collect all 2023 change subset predictions
df23_all_preds = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = combined.iloc[train_idx]
    test_data = combined.iloc[test_idx]

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)
    print(i , groups_all.iloc[test_idx].unique())

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

    # Get pseudo labels for 2023 change points
    df23changesubset = gdf23[(gdf23['change'] == 1) & (gdf23['cluster']==groups_all.iloc[test_idx].unique()[0])].copy()
    X = df23changesubset[wavelength23_cols]
    predicted_labels = clf.predict(X)
    df23changesubset['predicted_class'] = predicted_labels
    df23_all_preds.append(df23changesubset)
    print(df23changesubset['predicted_class'].value_counts())

df23_all_preds = pd.concat(df23_all_preds, ignore_index=False)

# Summary
print(f"\nAverage Accuracy (only folds using 2023 test data): {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")


0 [3]

Fold 1 — Accuracy: 0.50, F1 Score: 0.50
              precision    recall  f1-score   support

           0       0.46      0.46      0.46        24
           1       0.00      0.00      0.00         0
           2       0.23      0.49      0.31        47
           3       0.42      0.68      0.52       142
           4       0.54      0.71      0.61       123
           5       0.77      0.28      0.41        72
           6       0.80      0.39      0.52       103
           7       0.75      0.54      0.63        74
           8       0.64      0.29      0.40       181
           9       0.58      0.76      0.66        25
          10       0.48      1.00      0.65        16

    accuracy                           0.50       807
   macro avg       0.52      0.51      0.47       807
weighted avg       0.59      0.50      0.50       807

predicted_class
4    37
8     8
3     8
7     6
0     4
1     2
5     1
9     1
Name: count, dtype: int64
1 [0]

Fold 2 — Accuracy: 0.52, F1

In [55]:
df23changesubset['class'] = df23changesubset['predicted_class'].astype(int)
dfallsemisuperivised = pd.concat([gdf18[['class', 'cluster']+wavelength23_cols], df23subset[['class', 'cluster']+wavelength23_cols], df23changesubset[['class', 'cluster']+wavelength23_cols]], ignore_index=True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import pandas as pd
import numpy as np

# Combine for consistent group splitting
X_all = dfallsemisuperivised[wavelength23_cols]
y_all = dfallsemisuperivised['class'].astype(int)
groups_all = dfallsemisuperivised['cluster']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
clf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Metrics
f1_scores = []
accuracy_scores = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = dfallsemisuperivised.iloc[train_idx]
    test_data = dfallsemisuperivised.iloc[test_idx]

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy: {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")



Fold 1 — Accuracy: 0.51, F1 Score: 0.52
              precision    recall  f1-score   support

           0       0.48      0.46      0.47        24
           1       0.00      0.00      0.00         0
           2       0.23      0.47      0.31        47
           3       0.42      0.69      0.52       142
           4       0.54      0.72      0.62       123
           5       0.78      0.35      0.48        72
           6       0.81      0.41      0.54       103
           7       0.73      0.50      0.59        74
           8       0.70      0.31      0.44       181
           9       0.59      0.76      0.67        25
          10       0.50      1.00      0.67        16

    accuracy                           0.51       807
   macro avg       0.53      0.51      0.48       807
weighted avg       0.61      0.51      0.52       807


Fold 2 — Accuracy: 0.51, F1 Score: 0.53
              precision    recall  f1-score   support

           0       0.90      0.45      0.60       

In [45]:
# get pseudo-labels for 2023 change class

df23changesubset = gdf23[gdf23['change'] == 1].copy()
X = df23changesubset[wavelength23_cols]
predicted_labels = clf.predict(X)
df23changesubset['predicted_class'] = predicted_labels

# Ensure the 'class' column is of compatible type (e.g., int)
df23changesubset['class'] = df23changesubset['class'].astype(int)

# Compare predicted and labels before change (lower agreement is better)
matches = df23changesubset['predicted_class'] == df23changesubset['class']

# Compute proportion (i.e., accuracy)
proportion = matches.mean()

print(f"Proportion of matches: {proportion:.2f}")

Proportion of matches: 0.17


In [ ]:
# without spatial CV
df1823 = pd.concat([gdf18[['class']+wavelength23_cols], df23subset[['class']+wavelength23_cols]], ignore_index=True)
df1823

n18 = len(gdf18)
n23 = len(df23subset)


In [67]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

# Load data
X, y = df1823[wavelength23_cols], df1823['class'].astype(int)
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(X, y, df1823.index, test_size=0.5, random_state=42)

clf.fit(X_train, y_train)

RandomForestClassifier()

In [68]:
train_from_18 = (idx_train < n18).sum()
train_from_23 = (idx_train >= n18).sum()
test_from_18 = (idx_test < n18).sum()
test_from_23 = (idx_test >= n18).sum()

train_total = len(idx_train)
test_total = len(idx_test)

print("Train proportions:")
print(f"df18: {train_from_18 / train_total:.2f}, df23: {train_from_23 / train_total:.2f}")

print("Test proportions:")
print(f"df18: {test_from_18 / test_total:.2f}, df23: {test_from_23 / test_total:.2f}")


Train proportions:
df18: 0.55, df23: 0.45
Test proportions:
df18: 0.55, df23: 0.45


In [69]:
prediction_probabilities = clf.predict_proba(X_test)
prediction_probabilities.shape

# Get the predicted class labels
predicted_labels = clf.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

# from .63

Accuracy: 0.65


In [70]:
# get pseudo-labels for 2023 change class

df23changesubset = gdf23[gdf23['change'] == 1].copy()
X = df23changesubset[wavelength23_cols]
predicted_labels = clf.predict(X)
df23changesubset['predicted_class'] = predicted_labels

# Ensure the 'class' column is of compatible type (e.g., int)
df23changesubset['class'] = df23changesubset['class'].astype(int)

# Compare predicted and true labels
matches = df23changesubset['predicted_class'] == df23changesubset['class']

# Compute proportion (i.e., accuracy)
proportion = matches.mean()

print(f"Proportion of matches: {proportion:.2f}")


Proportion of matches: 0.15


In [71]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(df23changesubset['class'], df23changesubset['predicted_class']))
print(classification_report(df23changesubset['class'], df23changesubset['predicted_class']))


[[ 4  0  0  0  0  0  0  0  0  0 10]
 [ 0  0  0  0  0  0  0  0  3  0  0]
 [ 4  0  8 12 36  8  4  1 10  0  0]
 [ 1  0  3  6 11  1  0  0 11  0  0]
 [ 6  0  1  3 22  0  0  1  1  0  0]
 [ 1  0  0  1  1  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  3  0  0  0  0  0  0]
 [ 0  0  0  7 80  0  0 18  9  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  2]]
              precision    recall  f1-score   support

           0       0.25      0.29      0.27        14
           1       0.00      0.00      0.00         3
           2       0.67      0.10      0.17        83
           3       0.20      0.18      0.19        33
           4       0.14      0.65      0.24        34
           5       0.00      0.00      0.00         3
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         4
           9       1.00      0.01      0.02       115
        

In [72]:
df23changesubset['class'] = df23changesubset['predicted_class'].astype(int)
dfallsemisuperivised = pd.concat([gdf18[['class']+wavelength23_cols], df23subset[['class']+wavelength23_cols], df23changesubset[['class']+wavelength23_cols]], ignore_index=True)

In [73]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

# Load data
X, y = dfallsemisuperivised[wavelength23_cols], dfallsemisuperivised['class'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

clf.fit(X_train, y_train)

RandomForestClassifier()

In [74]:
# Get the predicted class labels
predicted_labels = clf.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

# from .65

Accuracy: 0.67


TABPFN

In [57]:
# with spatial CV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import pandas as pd
import numpy as np

# Add source labels
gdf18['source'] = '2018'
df23subset['source'] = '2023'

# Combine for consistent group splitting
combined = pd.concat([gdf18, df23subset], ignore_index=True)
X_all = combined[wavelength23_cols]
y_all = combined['class'].astype(int)
groups_all = combined['cluster']

# Track source to separate later
sources = combined['source']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)

# Metrics
f1_scores = []
accuracy_scores = []

# Collect all 2023 change subset predictions
df23_all_preds = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = combined.iloc[train_idx]
    test_data = combined.iloc[test_idx]

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)
    print(i , groups_all.iloc[test_idx].unique())

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

    # Get pseudo labels for 2023 change points
    df23changesubset = gdf23[(gdf23['change'] == 1) & (gdf23['cluster']==groups_all.iloc[test_idx].unique()[0])].copy()
    X = df23changesubset[wavelength23_cols]
    predicted_labels = clf.predict(X)
    df23changesubset['predicted_class'] = predicted_labels
    df23_all_preds.append(df23changesubset)
    print(df23changesubset['predicted_class'].value_counts())

df23_all_preds = pd.concat(df23_all_preds, ignore_index=False)

# Summary
print(f"\nAverage Accuracy (only folds using 2023 test data): {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")


0 [3]


100%|██████████| 16/16 [02:41<00:00, 10.11s/it]



Fold 1 — Accuracy: 0.65, F1 Score: 0.66
              precision    recall  f1-score   support

           0       0.78      0.58      0.67        24
           1       0.00      0.00      0.00         0
           2       0.32      0.64      0.43        47
           3       0.57      0.73      0.64       142
           4       0.67      0.71      0.69       123
           5       0.82      0.46      0.59        72
           6       0.87      0.71      0.78       103
           7       0.85      0.77      0.81        74
           8       0.74      0.49      0.59       181
           9       0.64      0.92      0.75        25
          10       0.62      1.00      0.76        16

    accuracy                           0.65       807
   macro avg       0.62      0.64      0.61       807
weighted avg       0.70      0.65      0.66       807



100%|██████████| 16/16 [02:11<00:00,  8.19s/it]


predicted_class
4    36
7    10
3     7
8     7
2     4
1     2
5     1
Name: count, dtype: int64
1 [0]


...

predicted_class
4    37
8    10
3     7
7     3
0     2
5     1
Name: count, dtype: int64
4 [1]


100%|██████████| 16/16 [03:02<00:00, 11.42s/it]



Fold 5 — Accuracy: 0.63, F1 Score: 0.63
              precision    recall  f1-score   support

           0       0.82      0.82      0.82        28
           1       0.50      0.04      0.08        45
           2       0.69      0.53      0.60        34
           3       0.81      0.71      0.75        82
           4       0.61      0.66      0.63        65
           5       0.07      0.33      0.11         6
           6       0.14      0.50      0.22        10
           7       0.66      0.75      0.70        60
           8       0.56      0.61      0.58        56
           9       0.86      0.67      0.75        48
          10       0.94      1.00      0.97        32

    accuracy                           0.63       466
   macro avg       0.61      0.60      0.57       466
weighted avg       0.68      0.63      0.63       466



100%|██████████| 16/16 [02:39<00:00,  9.94s/it]

predicted_class
4     39
8      7
7      7
3      3
10     2
5      2
Name: count, dtype: int64

Average Accuracy (only folds using 2023 test data): 0.68 (0.04)
Average F1 Score: 0.68 (0.04)


In [58]:
df23changesubset['class'] = df23changesubset['predicted_class'].astype(int)
dfallsemisuperivised = pd.concat([gdf18[['class', 'cluster']+wavelength23_cols], df23subset[['class', 'cluster']+wavelength23_cols], df23changesubset[['class', 'cluster']+wavelength23_cols]], ignore_index=True)

In [59]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, classification_report, f1_score
import pandas as pd
import numpy as np

# Combine for consistent group splitting
X_all = dfallsemisuperivised[wavelength23_cols]
y_all = dfallsemisuperivised['class'].astype(int)
groups_all = dfallsemisuperivised['cluster']

# GroupKFold
n_splits = 5
gkf = GroupKFold(n_splits=n_splits)

# Classifier
# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)

# Metrics
f1_scores = []
accuracy_scores = []

for i, (train_idx, test_idx) in enumerate(gkf.split(X_all, y_all, groups_all)):
    # Get train/test data
    train_data = dfallsemisuperivised.iloc[train_idx]
    test_data = dfallsemisuperivised.iloc[test_idx]

    X_train = train_data[wavelength23_cols]
    y_train = train_data['class'].astype(int)
    X_test = test_data[wavelength23_cols]
    y_test = test_data['class'].astype(int)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    accuracy_scores.append(acc)
    f1_scores.append(f1)

    print(f"\nFold {i+1} — Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
    print(classification_report(y_test, y_pred))

# Summary
print(f"\nAverage Accuracy: {np.mean(accuracy_scores):.2f} ({np.std(accuracy_scores):.2f})")
print(f"Average F1 Score: {np.mean(f1_scores):.2f} ({np.std(f1_scores):.2f})")


100%|██████████| 16/16 [02:56<00:00, 11.00s/it]



Fold 1 — Accuracy: 0.65, F1 Score: 0.66
              precision    recall  f1-score   support

           0       0.82      0.58      0.68        24
           1       0.00      0.00      0.00         0
           2       0.35      0.64      0.45        47
           3       0.57      0.71      0.63       142
           4       0.65      0.70      0.67       123
           5       0.78      0.44      0.57        72
           6       0.89      0.75      0.81       103
           7       0.82      0.72      0.76        74
           8       0.75      0.52      0.62       181
           9       0.61      0.88      0.72        25
          10       0.59      1.00      0.74        16

    accuracy                           0.65       807
   macro avg       0.62      0.63      0.61       807
weighted avg       0.70      0.65      0.66       807



100%|██████████| 16/16 [03:06<00:00, 11.64s/it]



Fold 2 — Accuracy: 0.66, F1 Score: 0.67
              precision    recall  f1-score   support

           0       0.93      0.65      0.76        20
           1       0.00      0.00      0.00         0
           2       0.86      0.65      0.74       192
           3       0.34      0.43      0.38        70
           4       0.72      0.64      0.68        81
           5       0.62      0.66      0.64        91
           6       0.62      0.80      0.70        44
           7       0.80      0.91      0.85        22
           8       0.55      0.61      0.58        97
           9       0.00      0.00      0.00         0
          10       1.00      0.95      0.97        39

    accuracy                           0.66       656
   macro avg       0.59      0.57      0.57       656
weighted avg       0.70      0.66      0.67       656



100%|██████████| 16/16 [03:09<00:00, 11.81s/it]



Fold 3 — Accuracy: 0.70, F1 Score: 0.69
              precision    recall  f1-score   support

           0       0.74      0.70      0.72        46
           1       0.00      0.00      0.00        38
           2       0.40      0.78      0.53        32
           3       0.88      0.71      0.79       201
           4       0.69      0.82      0.75       125
           5       0.49      0.61      0.54        28
           6       0.44      0.57      0.50        28
           7       0.71      0.75      0.73        48
           8       0.60      0.50      0.55        18
           9       0.00      0.00      0.00         0
          10       1.00      0.98      0.99        41

    accuracy                           0.70       605
   macro avg       0.54      0.58      0.55       605
weighted avg       0.70      0.70      0.69       605



100%|██████████| 16/16 [03:13<00:00, 12.07s/it]



Fold 4 — Accuracy: 0.74, F1 Score: 0.75
              precision    recall  f1-score   support

           0       0.91      0.62      0.74        16
           1       0.00      0.00      0.00         0
           2       0.87      0.66      0.75       148
           3       0.83      0.76      0.79       126
           4       0.72      0.78      0.75        76
           5       0.09      0.25      0.13         4
           6       0.63      0.65      0.64        52
           7       0.62      0.73      0.67        22
           8       0.48      0.82      0.60        38
           9       0.88      0.88      0.88        42
          10       0.96      1.00      0.98        24

    accuracy                           0.74       548
   macro avg       0.63      0.65      0.63       548
weighted avg       0.78      0.74      0.75       548



100%|██████████| 16/16 [03:11<00:00, 11.98s/it]


Fold 5 — Accuracy: 0.69, F1 Score: 0.69
              precision    recall  f1-score   support

           0       0.79      0.82      0.81        28
           1       0.67      0.09      0.16        45
           2       0.67      0.53      0.59        34
           3       0.81      0.72      0.76        85
           4       0.76      0.81      0.79       104
           5       0.12      0.50      0.20         8
           6       0.15      0.50      0.23        10
           7       0.75      0.78      0.76        67
           8       0.62      0.70      0.66        63
           9       0.84      0.67      0.74        48
          10       0.94      1.00      0.97        34

    accuracy                           0.69       526
   macro avg       0.65      0.65      0.61       526
weighted avg       0.74      0.69      0.69       526


Average Accuracy: 0.69 (0.03)
Average F1 Score: 0.69 (0.03)


In [75]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from tabpfn import TabPFNClassifier
from tabpfn_extensions.many_class import ManyClassClassifier

# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Load data
X, y = df1823[wavelength23_cols], df1823['class'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)
clf.fit(X_train, y_train)

ManyClassClassifier(alphabet_size=10, estimator=TabPFNClassifier())

In [76]:
# Predict probabilities
# prediction_probabilities = clf.predict_proba(X_test)
# print("ROC AUC:", roc_auc_score(y_test, prediction_probabilities[:, 1]))

# Predict labels
predictions = clf.predict(X_test)
print("Accuracy", accuracy_score(y_test, predictions))

100%|██████████| 16/16 [00:05<00:00,  2.75it/s]

Accuracy 0.7696301103179753


In [77]:
# get pseudo-labels for 2023 change class

df23changesubset = gdf23[gdf23['change'] == 1].copy()
X = df23changesubset[wavelength23_cols]
predicted_labels = clf.predict(X)
df23changesubset['predicted_class'] = predicted_labels

# Ensure the 'class' column is of compatible type (e.g., int)
df23changesubset['class'] = df23changesubset['class'].astype(int)

# Compare predicted and true labels
matches = df23changesubset['predicted_class'] == df23changesubset['class']

# Compute proportion (i.e., accuracy)
proportion = matches.mean()

print(f"Proportion of matches: {proportion:.2f}")


100%|██████████| 16/16 [00:04<00:00,  3.57it/s]

Proportion of matches: 0.22


In [78]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(df23changesubset['class'], df23changesubset['predicted_class']))
print(classification_report(df23changesubset['class'], df23changesubset['predicted_class']))


[[ 4  0  0  0  0  0  0  0  0  0 10]
 [ 0  2  0  0  0  0  0  0  1  0  0]
 [ 2  1 19 16 33  6  1  2  3  0  0]
 [ 0  0  5 13 11  1  0  0  3  0  0]
 [ 2  0  0  4 22  0  0  5  1  0  0]
 [ 0  1  1  0  1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  3  0  0  0  0  0  0]
 [ 0  0  0  2 87  0  0 16 10  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  2]]
              precision    recall  f1-score   support

           0       0.50      0.29      0.36        14
           1       0.50      0.67      0.57         3
           2       0.76      0.23      0.35        83
           3       0.36      0.39      0.38        33
           4       0.14      0.65      0.23        34
           5       0.00      0.00      0.00         3
           6       0.50      1.00      0.67         1
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         4
           9       0.00      0.00      0.00       115
        

In [79]:
df23changesubset['class'] = df23changesubset['predicted_class'].astype(int)
dfallsemisuperivised = pd.concat([gdf18[['class']+wavelength23_cols], df23subset[['class']+wavelength23_cols], df23changesubset[['class']+wavelength23_cols]], ignore_index=True)

In [80]:
# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Load data
X, y = dfallsemisuperivised[wavelength23_cols], dfallsemisuperivised['class'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)
clf.fit(X_train, y_train)


ManyClassClassifier(alphabet_size=10, estimator=TabPFNClassifier())

In [81]:
# Predict labels
predictions = clf.predict(X_test)
print("Accuracy", accuracy_score(y_test, predictions))

# from .786

100%|██████████| 16/16 [00:06<00:00,  2.35it/s]

Accuracy 0.7806757557794902


### Experiment 6: train on 2023 (stable), predict on 2018 (all labelled)

In [82]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

# Load data
X, y = df23subset[wavelength23_cols], df23subset['class'].astype(int)

clf.fit(X, y)

RandomForestClassifier()

In [85]:
prediction_probabilities = clf.predict_proba(gdf18[wavelength23_cols])
prediction_probabilities.shape

# Get the predicted class labels
predicted_labels = clf.predict(gdf18[wavelength23_cols])
# Calculate accuracy
accuracy = accuracy_score(gdf18['class'], predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

# from .04

Accuracy: 0.44


In [86]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from tabpfn import TabPFNClassifier
from tabpfn_extensions.many_class import ManyClassClassifier

# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Load data
X, y = df23subset[wavelength23_cols], df23subset['class'].astype(int)

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)
clf.fit(X, y)

ManyClassClassifier(alphabet_size=10, estimator=TabPFNClassifier())

In [89]:
# Get the predicted class labels
predicted_labels = clf.predict(gdf18[wavelength23_cols])
# Calculate accuracy
accuracy = accuracy_score(gdf18['class'], predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

# from .05

Accuracy: 0.50


### Experiment 7: train on 2018 (all) and 2023 (all), predict on 2023 (no change)

In [90]:
# use 2018 class for 2023 whereever there is no change
gdf23.loc[gdf23['change'] == 0, '2023_class'] = gdf23['class']
gdf23.dropna(subset=['2023_class'], inplace=True)

In [91]:
df1823 = pd.concat([gdf18[['class']+wavelength23_cols],
                    gdf23[['2023_class']+wavelength23_cols].rename(columns={'2023_class': 'class'})]
, ignore_index=True)
df1823

n18 = len(gdf18)
n23 = len(gdf23)


In [92]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

# Load data
X, y = df1823[wavelength23_cols], df1823['class'].astype(int)
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(X, y, df1823.index, test_size=0.5, random_state=42)

clf.fit(X_train, y_train)

RandomForestClassifier()

In [93]:
train_from_18 = (idx_train < n18).sum()
train_from_23 = (idx_train >= n18).sum()
test_from_18 = (idx_test < n18).sum()
test_from_23 = (idx_test >= n18).sum()

train_total = len(idx_train)
test_total = len(idx_test)

print("Train proportions:")
print(f"df18: {train_from_18 / train_total:.2f}, df23: {train_from_23 / train_total:.2f}")

print("Test proportions:")
print(f"df18: {test_from_18 / test_total:.2f}, df23: {test_from_23 / test_total:.2f}")


Train proportions:
df18: 0.49, df23: 0.51
Test proportions:
df18: 0.51, df23: 0.49


In [94]:
prediction_probabilities = clf.predict_proba(X_test)
prediction_probabilities.shape

# Get the predicted class labels
predicted_labels = clf.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

# from .59

Accuracy: 0.62


In [95]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from tabpfn import TabPFNClassifier
from tabpfn_extensions.many_class import ManyClassClassifier

# Create a base TabPFN classifier
base_clf = TabPFNClassifier()

# Load data
X, y = df1823[wavelength23_cols], df1823['class'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Initialize a classifier
clf = ManyClassClassifier(
    estimator=base_clf,
    alphabet_size=10  # Use TabPFN's maximum class limit
)
clf.fit(X_train, y_train)

ManyClassClassifier(alphabet_size=10, estimator=TabPFNClassifier())

In [96]:
# Predict probabilities
# prediction_probabilities = clf.predict_proba(X_test)
# print("ROC AUC:", roc_auc_score(y_test, prediction_probabilities[:, 1]))

# Predict labels
predictions = clf.predict(X_test)
print("Accuracy", accuracy_score(y_test, predictions))

# from .748

100%|██████████| 16/16 [00:06<00:00,  2.43it/s]

Accuracy 0.7291049199762892
